### 작업전(t일 이전) 할 거 : 계좌다운전 오류수정,항고건,12월연납건,다운시 진행내용표시
0. 조회 > 변제오류까지 작업 > 변제오류건 최신화 누른 후 다운로드 다시

1. 항고건 살피기 : 아무나 한 건 검색, 아무셀이나 필터적용해서 CUSTOMIZE단추 열기, 필터함수 불러오고 원래 검색조건은 삭제 후 검색(D:\3.자산\전산 dataset)
  - 건수 3건 이상이면 업데이트는 엑셀로 하는 게 편한듯.
2. 12월에 일반회생(연납건) 입금회차 확인
[20420482 보증인]
[20420209 차주]
[n1027937 : 20426827,20426904 차주]
입금월 12, 12, 12, 12

- 수정대상건에 대해 전산수정작업 마친 후 t+1일에 다운로드()
1. 우클릭 후 '1.채권자목록,일반내용,진행내용 표시'선택할 것!!

- 기준일 개회 입금 t+1에 다운로드. 계좌별 표시 x !!!!!!!!!!!!!!!!!!!!!!!!!!

### import 및 기본변수

In [20]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
from datetime import timedelta
import calendar
from dateutil.relativedelta import relativedelta
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, Font, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import functions
from pandas.tseries.offsets import DateOffset

# 파일 읽기
path_base = r"D:\3.자산\전산 dataset"
########################################################
company = "솔림"
cutoff = "241024" # 폴더명
previous_cutoff = "240108" # 폴더명
ext = ".pkl"   ####-------------------------------
# name_dataset = "개인회생새창_20240830_1653" 
########################################################

path_wd = join(path_base, company, cutoff)
path_wd_check = join(path_base, company, cutoff, "개인회생점검")
if not os.path.exists(path_wd_check) : os.mkdir(path_wd_check)

name_dataset = functions.키워드로파일명찾기(path_wd, "개인회생새창", "기준데이터", 전체경로=False).split(".")[0]
path_result = join(path_wd, name_dataset+"_기준데이터.xlsx")

rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str, '신고계좌':str, '입금계좌':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}

# ------------------------------------------------
검색용사명 = "(솔림|헬프)" if company=="솔림" else "대성인베스트"
신고채권자사명 = "(솔림|헬프|엔에)" if company=="솔림" else "대성" #######################

cutoff_dt = datetime.strptime(cutoff, "%y%m%d")
조회일_dt = datetime.strptime(name_dataset.split("_")[1], "%Y%m%d")
조회일 = datetime.strftime(조회일_dt, "%Y-%m-%d")

# ------------------------------------------------
# 금지명령(중지명령) : 전산은 금지명령이 기각되는 경우를 무시하고 신청서제출만으로 금지로 처리함.
# 접수 : 금지명령신청만 한 경우나, 기각된 경우를 따로 표시하기 위해 추가함
항고인용사건_현재결과종류 = {"개시":"개인회생절차개시결정\x7f", "금지": r"금지명령\(|중지명령\x7f", # r"(금지|중지)명령신청서? 제출\x7f", 
          "면책허가결정":"면책허가결정공고\x7f", "신청취하" : "종국 : 신청취하\x7f", "이송" : "종국 : 이송\x7f", "인가" : r"(회생|변제)계획인가결정\x7f","인용" : "종국 : 인용\x7f",
          "폐지취소결정" :"폐지취소결정\x7f", "폐지취소결정송달" : "폐지취소결정 송달\x7f"}

기각폐지항고 = {"기각":"종국 : 기각\x7f", "기각결정":"개인회생절차개시신청 기각결정\x7f", 
          "폐지결정" : "폐지결정\(", "폐지":"종국 : 폐지\x7f", 
          "항고" : "항고장"}

현재결과종류 = {key: value for d in [항고인용사건_현재결과종류, 기각폐지항고] for key, value in d.items()}

def 현재결과찾기(진행내용:str) -> str:
    # 진행내용 최근 결정내용을 찾아 현재결과 리턴
    for value in reversed(진행내용.split("\n")) :
        for i, v in 현재결과종류.items() :
            if re.search(v, value) :
                return i
            
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

def 항고사건_현재결과찾기(진행내용:str) -> str :
    for value in reversed(진행내용.split("\n")) :
        for i, v in 항고인용사건_현재결과종류.items() :
            if re.search(v, value) :
                return i
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면


def 말일오류없애기(치환할날:datetime, 변제기일:int):
    """2024-02-31을 2024-02-29로"""
    마지막날 = calendar.monthrange(치환할날.year, 치환할날.month)[1]
    if 변제기일 > 마지막날:
        return datetime(치환할날.year, 치환할날.month, 마지막날)
    else:
        return datetime(치환할날.year, 치환할날.month, 변제기일)
                

def 특정날짜후첫변제일(reference_date:datetime, 변제기일:int) :
    년도 = reference_date.year
    월 = reference_date.month
    날짜 = reference_date.day
    
    if 날짜 >= 변제기일 : # 컷오프 다음날부터 당사 자산
        if 월 == 12 :
            월 = 1
            년도 += 1
        else : 
            월 += 1
            
    첫날 = datetime(년도, 월, 1)
    return 말일오류없애기(첫날, 변제기일)
        


# 채권상태분류
comp_closing = {
    # 채권 자체의 소멸
    "종료" : r"(?<!청산)종결(?!요청)|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료",
    # 비정상-종결요청
    "종결요청" : r"종결요청|매각\(예정\)|(?<!피)환매\(예정\)|패소",
    # 시효
    "시효" : r"시효|완성|만료", # 부활 제외함 231222
    # 채무자별로 체크
    "면책" : r"면책|면탈",  
    "불가" : r"추심제외|상각|포기|파산(?!(\(기각\)|\s누락))|사망(?!\(상속승인\))|폐업|청산종결|패소", # 상각, 제외, 포기는 이미 보증인까지 고려한 결과일 수 있음 # 담당자 누락제외는 채무상태에 적용하면 안 됨.(계좌의 메모를 봐야함)
    "공채권" : r"공채권"
    # 이민, 말소, 실종은 정상으로
    }

종료 = re.compile(comp_closing["종료"])
종결요청 = re.compile(comp_closing["종결요청"])
시효 = re.compile(comp_closing["시효"])
면책 = re.compile(comp_closing["면책"])
불가 = re.compile(comp_closing["불가"])
공채권 = re.compile(comp_closing["공채권"])

# 칼럼정의
기본조회칼럼 = ["채무자키","계좌키","분납키","분납자관계","분납자성명인","주민번호인","관할법원","사건번호", "인가미인가", "현재결과", "분납메모","분납기타","분납비고", "기타금액"]
핵심금액칼럼 = ["총분납금", "총분납입금", "총분납잔금"]

전산회차칼럼 = ["총분납회차","현재회차", "납입회차", "미납회차"] #"초과입금" "잔여회차"
법원회차칼럼 = ["전체변제회차","현재변제회차", "법원납입회차", "법원미납회차", ] #"잔여변제회차"
회차보조칼럼 = ["현재회차차이", "납입회차차이", "채무자미납회차","출금대상회차"] #"매월변제예정금액","출금대상잔액"
변제분석칼럼 = ["입금컷오프일", "변제기일", "초회여부","최근변제예정일", "채권자변제시작일","당사변제시작일", "채권자변제시작회차"] # 실변제일과 변제기일, 실변제회차 동일(말일제외)

변제조회칼럼 = ["변제현황조회","신고조회일","신고조회결과"]
은행칼럼 = ["신고은행","신고계좌","신고채권자","입금은행", "입금계좌", "입금채권자"]
채권번호칼럼 = ["변제채권번호입력", "변제채권번호스캔", "변제채권번호"]

# 회차정보칼럼 = ["채권자종료회차", "변제시작일", "현재회차변제금", "말일여부"] "현재회차변제일"
회차구간칼럼 = ["변제회차1", "변제기간1", "변제회차2", "변제기간2", "변제회차3", "변제기간3", "회차수정오류여부"]
보조칼럼 = ["상태및담당자", "OPB","총분납금", "채권구분", "매각사구분"]

### 파일 읽기 및 저장

In [3]:
# 전달 미종결인가건 : 변제금 변화 등 확인 위해
전달미종결인가건 = pd.read_pickle(functions.키워드로파일명찾기(join(path_base, company, previous_cutoff, "개인회생점검"), "미종결인가건.*pkl", "", 전체경로=True))

# 이달 개인회생파일
if ext == ".xlsx" : 
    path_raw_data = join(path_wd, name_dataset+ext)
    raw_data_ori = pd.read_excel(path_raw_data, dtype=rehabilitation_dtype).fillna("")

    if raw_data_ori.loc[0,"진행내용"] == "" :
        print("개인회생 엑셀다운로드시 우클릭후 '1.채권자목록,일반내용,진행내용 표시'를 선택해야 합니다.")
    else : 
        pass

    # 사건번호 정렬
    raw_data_ori.sort_values(by=["관할법원", "사건번호"], inplace=True)
                
    # pkl 저장
    if not os.path.exists(join(path_wd, "pkl")) : 
        os.mkdir(join(path_wd, "pkl"))
    raw_data_ori.to_pickle(join(path_wd, "pkl", name_dataset+".pkl"))

# pkl 읽기    
else : 
    path_raw_data = join(path_wd, "pkl", name_dataset+ext)
    # path_raw_data = join(path_wd, "pkl", name_dataset+"_진행중_계좌키분리_기준데이터"+ext)
    raw_data_ori = pd.read_pickle(path_raw_data)


# 작업 전 확인 ----------------------------------------------------------------------------------------
raw_data_ori["상태및담당자"] = raw_data_ori["채무상태"] + raw_data_ori["담당자"]
# 기본조건
기본cond = (raw_data_ori.분납상태=="진행") & (~raw_data_ori["분납메모"].str.contains('제외|통합|일반')) & (~raw_data_ori.상태및담당자.str.contains(종료))
# 항고사건, 이유없는 기타금액 확인
항고cond = 기본cond & (raw_data_ori.현재결과=="항고") & (raw_data_ori.분납메모=="")
# 이유없는 기타금액
기타금액cond = 기본cond & (raw_data_ori.기타금액>0) & ((~raw_data_ori.분납기타.str.contains('총분납금추가'))&(~raw_data_ori.분납비고.str.contains('총분납금')))

# 변제계획안보고 총분납금을 분납비고에 작성할 것
변제계획안cond = 기본cond & (raw_data_ori["현재결과"] == "인가") & (raw_data_ori["신고조회결과"].str.contains("은행코드")) & (raw_data_ori["총분납금"] == 0) & (~raw_data_ori["분납비고"].str.contains('총분납금_[\d,]+', regex=True))

print('항고사건결과 또는 기타금액의 사유 또는 총분납금을 작성할것')
raw_data_ori.loc[항고cond|기타금액cond|변제계획안cond, ["채무자키", "분납키", "분납자성명인", "현재결과", "분납메모", "분납기타", "분납비고", "기타금액", "신고조회결과"]]

항고사건결과 또는 기타금액의 사유 또는 총분납금을 작성할것


,채무자키,분납키,분납자성명인,현재결과,분납메모,분납기타,분납비고,기타금액,신고조회결과


In [14]:
# # 바로 수정
# raw_data_ori.loc[raw_data_ori["분납키"]=="20137408", "분납메모"] = "미결정, 2024-10-22"

### 재시작점-----------

In [4]:
raw_data = raw_data_ori.copy()

# 특이건, 법원전산오류건 수정(자산정리전 확인) ###############
# 솔림,대성 개별적용건
if company == "솔림" : 
    # 채무자키 20410976,20410977 : 20회차와 21회차 사이에 9개월 간격있는데 법원전산이 간격을 간과하고 있음
    raw_data.loc[raw_data["분납키"]=="20107061", ["현재변제회차", "잔여변제회차", "채무자미납회차"]] += [-9, 9, -9]
    raw_data.loc[raw_data["분납키"]=="20107062", ["현재변제회차", "잔여변제회차", "채무자미납회차"]] += [-9, 9, -9]

else : 
    pass

# 공통적용가능 코드
# 1) 매입OPB수정건
def extract_opb(value):
    match = re.search(r'매입OPB_([\d,]+)', value)
    if match:
        # 쉼표 제거하고 숫자로 변환
        return int(match.group(1).replace(',', ''))
    return None

    # '분납기타' 열에서 '매입OPB_' 패턴을 찾아 'OPB' 열 업데이트
raw_data['OPB'] = raw_data.apply(
    lambda row: extract_opb(row['분납기타']) if extract_opb(row['분납기타']) is not None else row['OPB'],
    axis=1
)

# 2-1) 총분납금 : 기타금액추가
# 스케줄 미반영분 : 초회, 미등록채권 : 잔여금액 총분납잔금에 포함시켜야 하므로 총분납금에도 추가!(해당하는 만큼 초과입금 삭제필요)
# 채권자변경전(법원에서 채권자변경후만 변제현황에 잡은 경우) -> 회생폐지 & 총분납금오류아님 으로 처리. 기타금액으로 처리할 시 총분납잔금과 무관한데, 괜히 초과입금만 잡혀서 별로
raw_data["총분납금"] += raw_data["기타금액"] 



# 2-2) 총분납금 : 스케줄(변제조회) 실패건, 변제계획안 보고 분납비고에 총분납금 적을 것 ---------------------
    # 분납기타 양식 : 총분납금_123,456,789
    # cf)변제현황서비스이용불가건이라도 정상적인 건은 스케줄 잡힘

총분납금cols = ["채무자키","분납키","분납메모","분납비고","기타금액","총분납금", "변제현황조회","신고조회결과"]

    # 필터링 조건 설정
인가cond = (raw_data["분납상태"] == "진행")&(~raw_data["분납메모"].str.contains('제외|통합|일반')) & (~raw_data_ori.상태및담당자.str.contains(종료))& (raw_data["변제계획인가일"] != "")
변제현황cond = raw_data["변제현황조회"].str.contains('조회') # 변제현황조회 서비스를 이용할 수 없습니다. # 변제현황조회성공인데 총분납금 0인 경우는 아마 별제권일 것이다. 그외에 다른 경우는 없는 것을 확인함
총분납금cond = raw_data["총분납금"] == 0
    
총분납금재작성cond = 인가cond & 변제현황cond & 총분납금cond
분납비고삭제cond = 인가cond & ~총분납금cond & raw_data["분납비고"].str.contains('총분납금_\d') # 이제 조회 돼서 총분납금 0 아닌 건

    # '분납비고' 열에서 '총분납금_숫자' 패턴에 맞는 숫자 추출
raw_data.loc[총분납금재작성cond, '총분납금_추출'] = raw_data.loc[총분납금재작성cond, '분납비고'].str.extract(r'총분납금_([\d,]+)').iloc[:, 0].str.replace(',', '')

    # 오류 및 분납비고 삭제건 --------------------------
출력1 = raw_data.loc[총분납금재작성cond & raw_data['총분납금_추출'].isna(), 총분납금cols]
출력1_1 = raw_data.loc[인가cond & 총분납금cond & ~raw_data["분납비고"].str.contains('별제권|미확정') & raw_data['총분납금_추출'].isna(), 총분납금cols]
if len(출력1) > 0 :
    print('총분납금 입력해줘야할 건')
    display(출력1)
if len(출력1_1) > 0 :
    print('총분납금 입력해줘야할 건2 : 변제현황 읽어지는데 최신화가 안 돼서 스케줄 안 잡힌 경우')
    display(출력1_1)
    
else : 
    print('총분납금 입력해줘야할 건 --- 없음')

출력2 = raw_data.loc[분납비고삭제cond, 총분납금cols]
if len(출력2) > 0 :
    print('변제현황 정상 작동해서 비고 삭제할 건') # 총분납금0에만 비고값을 넣어주므로 비고삭제건에 총분납금 중복반영되는 것은 아니다.
    display(출력2)
else : 
    print('변제현황 정상 작동해서 비고 삭제할 건 --- 없음')
    # ------------------------------------------------
    
    # 추출한 값이 있는 경우, 해당 값을 '총분납금' 열에 숫자로 변환하여 대입
    # ori파일에서 이미 확인할 거 출력했지만, 혹여나 추출결과가 빈값인 경우 이 코드에서 에러남
총분납금추가할ind = raw_data[총분납금재작성cond & raw_data['총분납금_추출'].notna()].index
raw_data.loc[총분납금추가할ind, '총분납금'] = raw_data.loc[총분납금추가할ind, '총분납금_추출'].astype(int)

    # '총분납금_추출' 열 제거 (필요시)
raw_data.drop(columns=['총분납금_추출'], inplace=True)


# 칼럼추가
# 최초OPB : 매입OPB말고 최초원금 + 최초법비용
raw_data["최초OPB"] = raw_data["최초원금"] + raw_data["최초법비용"]


# 확인사례
# 총분납금 확인 : 최근인가라서 스케줄 안 잡힌거는 분납비고에만 "총분납금_1,478,280"이런 식으로 작성해준다
# print('분납비고 금액이 총분납금에 잘 입력되었는지 확인')
# display(raw_data.loc[총분납금추가할ind, 총분납금cols])

총분납금 입력해줘야할 건 --- 없음
변제현황 정상 작동해서 비고 삭제할 건 --- 없음


In [77]:
# # 총분납금 입력 수정할 거 있는 경우에만 실행 -------------------------------
# # 원파일과 복사본 바로 수정 하고
# # 엑셀 파일도 직접수정!!!!!!!!!!!!, ori에는 전산 수정한 칼럼만 저장
# ############################################################################################ 
# # '인가신고'라는 접두사는 붙이지 않는다. 여기서 삭제할 건 조사하니까! 안그러면 아래서 분납비고삭제하라고 뜸
# 수정할분납키 = "20148175"
# raw_data_ori.loc[raw_data_ori["분납키"]==수정할분납키,"분납비고"] = "총분납금_1,478,280"
# raw_data.loc[raw_data["분납키"]==수정할분납키,["분납비고","총분납금"]] = ["총분납금_1,478,280", 1478280]

# 수정할분납키 = "20156307"
# raw_data_ori.loc[raw_data_ori["분납키"]==수정할분납키,"분납비고"] = "총분납금_7209956"
# raw_data.loc[raw_data["분납키"]==수정할분납키,["분납비고","총분납금"]] = ["총분납금_7209956", 7209956]

# 수정할분납키 = "20104944"
# raw_data_ori.loc[raw_data_ori["분납키"]==수정할분납키,"분납비고"] = "변제내용_총분납금오류아님"
# raw_data.loc[raw_data_ori["분납키"]==수정할분납키,"분납비고"] = "변제내용_총분납금오류아님"
# ############################################################################################

# # pkl 저장
# if not os.path.exists(join(path_wd, "pkl")) : 
#     os.mkdir(join(path_wd, "pkl"))
# raw_data_ori.to_pickle(join(path_wd, "pkl", name_dataset+".pkl"))

### 최근 접수건만 남기기, 당사누락건 삭제하기

In [5]:
# 접수일 없는 건은 모두 중단건이다.(자동으로 불러오지 못한 거니까). 따라서 일단 제일 뒤로 보내기 위해 임의의 날짜값으로 대체
raw_data.loc[raw_data.query('접수일==""').index,"접수일"] = "1975-01-01"
# 역정렬
raw_data = raw_data.sort_values(by='접수일', ascending=False)

# 삭제할 것 중 분납상태 진행 있는지 체크
삭제할중복값 = raw_data[raw_data.duplicated(subset=['계좌키', '분납자성명인'], keep='first')]
if len(삭제할중복값.query('분납상태=="진행"')) : 
    display(삭제할중복값.query('분납상태=="진행"'))
else : 
    print(len(raw_data))
    raw_data.drop_duplicates(subset=['계좌키', '분납자성명인'], keep='first', inplace=True)
    raw_data.reset_index(drop=True, inplace=True)
    print(len(raw_data))
    

# 당사누락 아닌 것만 남기기!!
raw_data = raw_data[~raw_data['분납메모'].str.contains('당사\s?누락')]

9798
9344


### 사전점검 : 동일 사건
- 현재결과 및 분납상태 다른 거 있는지 
- 분납자가 다른 경우

In [35]:
동일사건 = raw_data.query('분납상태!="중단"').groupby(by=['관할법원', '사건번호'])
상이한현재결과 = 동일사건.filter(lambda x: x['현재결과'].nunique() != 1)
if len(상이한현재결과)>0:
    print("동일사건의 현재결과 다름")
    display(상이한현재결과.sort_values(["사건번호"])[["채무자키","분납키","주민번호인","분납자관계","분납자성명인","관할법원","사건번호","현재결과","분납상태"]])
else : 
    print('동일사건의 현재결과 다른 건 없음')
# 분납상태가 모두 중단이면 현재결과 달라도 상관없음

분납자다름 = 동일사건.filter(lambda x: x['주민번호인'].nunique() != 1)
if len(분납자다름) > 0 :
    print("동일사건의 분납자 다름")
    display(분납자다름.sort_values(["사건번호"])[["채무자키","분납키","주민번호인","분납자관계","분납자성명인","관할법원","사건번호","현재결과","분납상태"]])
else : 
    print("동일사건의 분납자 다른 건 없음")

동일사건의 현재결과 다른 건 없음
동일사건의 분납자 다른 건 없음


### 인가미인가 작성

In [6]:
# 컬럼 순서 바꾸기, 인트를 문자열로(타입때메 종료제거보다 나중에 나와야)
raw_data.insert(1, "분납키", raw_data.pop("분납키"))
raw_data.insert(7, "현재결과", raw_data.pop("현재결과"))
raw_data.insert(8, "분납상태", raw_data.pop("분납상태"))
raw_data.insert(9, "검수상태", raw_data.pop("검수상태"))

# 현재결과 종류
# 인가 : 인가
# 미인가 : 개시, 금지, 금지명령
# 폐지 : 절차폐지결정일, 폐지결정, 기각, 기각결정, 신청취하, 폐지
# 면책 : 인용, 면책허가결정, 면책,              * 종결은 면책목록에서 제외함
# (230131) 개편
# 변제계획인가일 확인 할 것 : 폐지취소결정송달, 폐지취소결정
# 분납메모, 변제계획인가일 확인 할 것

# 인가미인가
인가미인가 = []  ###################################### 현재결과가 하루 늦게 업데이트 되는 경우 잇음. 면책결정일이 있으면 면책으로 하는 코드 작성할까?
for i, v in tqdm(raw_data[["현재결과", "분납키", "분납메모", "변제계획인가일", "항고", "진행내용"]].iterrows(), total=len(raw_data.index)) :
    result, opb = "", 0
    
    결과 = v.현재결과
    # 현재결과에 따라 인가,미인가,폐지,면책,이송,체크 정하기
    # 인가
    if 결과 == "인가" or (결과 =="종결" and re.search("일반회생", v.분납메모)): # 종결은 수기등록 하는 거 아니면 회단사건에서만 확인되고 있다. 
        result = "인가"                 # 분납메모가 있으면 회단사건의 종결은 인가와 다름없다.(법원 절차만 종결됐다는 소리이므로). 분납메모가 없다면 체크로 빠지도록 한다. 완제됐다면 분납상태에서 걸리겄지.
        
    # 미인가
    elif re.search("개시|금지|접수", 결과) : 
        result = "미인가"
        
    # 폐지
    elif (re.search("폐지결정|기각|취하", 결과)) or (결과 == "폐지") : 
        result = "폐지"
        
    # 폐지취소 - 분납메모 미사용
    elif re.search("폐지취소", 결과) : 
        result = "미인가" if v.변제계획인가일=="" else "인가" # 날짜 타입으로 바꾸면서 수정 2407
            
    # 면책
    elif re.search("인용|면책|완제", 결과) : # 일부 채권사것만 완제하는 경우가 있다. 종결로 적지 않고 완제로 적을 것!!!
        result = "면책"
        
    # 이송 : 분납상태 중지 & 새사건 입력되었어야
    elif re.search("이송", 결과) :
        result = "이송"
        
    # 실사필요 : 항고인데 현재결과가 어디에도 해당하지 않을 때 or 새로운 현재결과가 나온 경우
    elif re.search("항고", 결과) : 
        try : 
            항고사건결과 = re.sub(" ", "", re.search(r'[가-힣]+\s?[가-힣]+', v.분납메모).group())
        except :
            항고사건결과 = ""
            
        채무자항고 = False if re.search("채권", v.항고) else True
        
        # 항고사건결과 아직 없음 : 체크
        if 항고사건결과 == "" : 
            result = "체크_항고사건결과없음"
            
        # 미결정 확인 완료 : 인가일 있는지에 따라
        elif re.search("미결정", 항고사건결과) : 
            result = "미인가" if v.변제계획인가일 == "" else "인가"
            
        # 항고 인용, 기각취소, 폐지취소 ----------------ㅁㅁㅁ 현재 항고사건없이 바로 결정나온것에 기각취소, 기각, 폐지취소는 확인되었으나 폐지(?!취소)는 확인이 안 되어 코드에 미반영상태
        elif re.search("환송|인용|기각취소|폐지취소", 항고사건결과) :
            if 채무자항고 :
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
            else : 
                result = "체크_채권자항고" # 채권자 항고에 대한 기각취소는 없어야하고, 파기환송과 인용은 아직 사례를 못 봐서 데이터 수집 필요
        # 취하, 기각, 각하
        elif re.search("취하|각하|기각(?!취소)", 항고사건결과) : # ----------------ㅁㅁㅁ 항고사건없이 항고장제출 후 폐지가 나오는 경우 있다면 추가해야
            if 채무자항고 : 
                result = "폐지"
            else : 
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
        # 분납메모에 엉뚱한 말이 적힌 경우
        else : 
            result = "체크_항고사건결과를 잘못적음"
    elif 결과=="" : # 접수만 해놓은 건
        result = "미인가"
    
    # 위의 어떤 경우에도 해당하지 않는 경우 : 없어야 함
    else : result = "체크_항목에 없는 현재결과"

    인가미인가.append(result)

raw_data.insert(7, "인가미인가", 인가미인가)
raw_data.insert(11, "반영OPB", None)

# 폐지취소 이전 파일에서 불러왔던 코드
# elif re.search("폐지취소", 결과) :  # 폐지취소결정송달 포함
#     temp = previous.query('분납키==@v.분납키')["인가미인가"].values
#     if len(temp) == 1 : result = temp[0]
#     else : result = "체크"


# 인가미인가 없는 건 확인
인가미인가없는건  = raw_data.query('인가미인가.str.contains("체크") and 분납상태=="진행"')
if len(인가미인가없는건) > 0 :
    print("인가미인가 없는 건")
    display(인가미인가없는건[["채무자키","계좌키","분납키","주민번호인","인가미인가","현재결과","분납메모","항고일","변제계획인가일"]])
else : 
    print("인가미인가 없는 건 없음")

100%|██████████| 9342/9342 [00:00<00:00, 44005.88it/s]

인가미인가 없는 건 없음


### 일반오류체크

In [ ]:
# # # 확인 후 수정 or pass                               ####################
# raw_data.loc[수작업idx,["인가미인가"]] = ["미인가"]
# raw_data.loc[수작업idx,["분납메모","인가미인가"]] = ["미결정", "미인가"]
#                                                       ####################

In [7]:
# check ----------------------------
cols = ["채무자키", "분납키", "분납자관계", "분납자성명인", "인가미인가", "현재결과","현재결과일", "분납상태", "분납메모","중단사유", "분납중단일", "채무상태", "담당자", "개시결정일", "변제계획인가일", "사건번호"]
check = raw_data[cols].copy()
# 인가미인가 : 인가,미인가 // 폐지,면책,이송 // 체크
# 분납상태 : 진행, 중단

# 1. 인가미인가와 분납상태의 불일치 -> 인가 or 미인가 = 분납상태 진행
# 1-1.진행건 vs 상태 중단 : 중단사유 있는지?
중단사유적어 = check.query('인가미인가.str.contains("인가|미인가") & 분납상태=="중단" & 중단사유 == ""')
if len(중단사유적어) > 0 :
    print(f"1-1.중단사유 명시 필요. {len(중단사유적어)}건 엑셀출력") # 현재결과는 중단사유가 아니라 바로 확인 불가
    functions.save_df_to_excel_underline(중단사유적어,join(path_wd_check, "[check]개인회생_중단사유없음_"+cutoff+".xlsx"), key_columns_no=6)
    
# 1-2. 중단건(인가/미인가 아닌것) vs 상태 진행
진행건_인가미인가오류 = check.query('분납상태=="진행" & ~인가미인가.str.contains("인가|미인가")')
if len(진행건_인가미인가오류) :
    print("1-2.진행건인데 인가나 미인가가 아닌 것 있음. 폐지 맞는지 확인(폐지결정 나더라도 바로 중단으로 하지 않는다. 2024-03~ / 인가미인가가 맞으면 됨.)")
    display(진행건_인가미인가오류.sort_values(by=['인가미인가']))

# 2. not중지건 중 인가전 현재결과 vs 변제계획인가일 있는 경우
미인가건임 = check.query('분납상태=="진행" and 인가미인가=="미인가" and 변제계획인가일!=""')    ############## 20149032 는 회단사건으로 인가일 표기위치가 다름
인가건임 = check.query('분납상태=="진행" and 인가미인가=="인가" and 변제계획인가일=="" and 분납키!="20149032" and ~(분납메모.str.contains("일반회생"))')
if len(미인가건임) > 0 :
    print("2.미인가건인데 왜 인가일이 있음?")
    display(미인가건임)

if len(인가건임) > 0 :
    print("인가건인데 왜 인가일이 없음?, 종결건, 확인하지 않은 일반회생은 그럴 수 있음(분납메모 작성필요)")
    display(인가건임)

# 3. not중지건 중 개시결정일 있는데 전단계인 경우
# (반대)왜인지는 몰라도 개시결정일을 못 읽는 경우가 1~2건 있네.. 
개시건임 = check.query('분납상태=="진행" and 개시결정일 != "" and 현재결과.str.contains("금지|접수")')
개시건인데 = check.query('분납상태=="진행" and 개시결정일 == "" and 현재결과.str.contains("개시") and 분납키!="20149032"')
if len(개시건임) > 0 :
    print("3.개시일 있는데 왜 전단계? - 항고건은 분납메모를 통해서 ")
    display(개시건임)
    
if len(개시건인데) > 0 :
    print("3.개시인데 왜 개시결정일 없음? 나우리 최신화 문제인듯")
    print("raw_data수정 : 현재결과일을 개시결정일에 추가해서 따로 할 건 없음")
    raw_data.loc[개시건인데.index, "개시결정일"] = 개시건인데["현재결과일"]
    display(개시건인데)

# 4. not중지건 일반회생(회단,간회단) 중 분납메모 없는 건
미확인일반회생 = check.query('사건번호.str.contains("회단") and 분납상태=="진행" and 현재결과=="인가" and ~(분납메모.str.contains("일반회생"))')
if len(미확인일반회생) > 0 :
    print("4.인가된 일반회생인데 메모 왜 없니")
    display(미확인일반회생)

# 항고, 폐지취소 건 중 분납중지건은 확인 완료 (240130)
# 최신화 불가건 : 항고 뿐임 (폐지취소는 최신화 됨, 폐지 후 과거의 인가/미인가 여부를 확인하는 작업이 필요할뿐)
# 담당자, 채무상태, 누락 건 등은 자산정리 후 해야 정확

1-1.중단사유 명시 필요. 14건 엑셀출력
1-2.진행건인데 인가나 미인가가 아닌 것 있음. 폐지 맞는지 확인(폐지결정 나더라도 바로 중단으로 하지 않는다. 2024-03~ / 인가미인가가 맞으면 됨.)


,채무자키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,현재결과일,분납상태,분납메모,중단사유,분납중단일,채무상태,담당자,개시결정일,변제계획인가일,사건번호
8252,20525107,20137113,채무자,한상남,면책,인용,2024-09-05,진행,,,,개인회생(면책),개인회생,2020-03-17,2020-12-14,2019개회1068014
7234,20452741,20120075,채무자,배광호,면책,인용,2024-10-24,진행,,,,개인회생(면책),개인회생,2020-09-10,2021-08-26,2020개회209725
5579,20514735,20131149,채무자,이미경,면책,면책허가결정,2024-03-05,진행,,,,개인회생(면책),개인회생(특이),2020-12-21,2021-02-23,2020개회232114
8251,20527564,20137124,채무자,한상남,면책,인용,2024-09-05,진행,,,,개인회생(면책),개인회생,2020-03-17,2020-12-14,2019개회1068014
7873,20515045,20130759,채무자,권은희,폐지,절차폐지결정일,2024-10-16,진행,,,,개인회생(폐지진행중),배정(대기),2020-08-07,2020-12-22,2019개회54777
7147,20526610,20137027,채무자,홍만호,폐지,폐지결정,2024-09-30,진행,,,,개인회생(폐지진행중),배정(대기),2020-10-23,2021-03-17,2020개회211844
6176,20452124,20121530,채무자,임재민,폐지,절차폐지결정일,2024-10-24,진행,제외,,,개인회생(폐지진행중),배정(대기),2020-11-17,2021-01-13,2020개회19977
6175,20410276,20105792,채무자,임재민,폐지,절차폐지결정일,2024-10-24,진행,,,,개인회생(폐지진행중),배정(대기),2020-11-17,2021-01-13,2020개회19977
5738,20410297,20105857,채무자,김가영,폐지,절차폐지결정일,2024-10-22,진행,,,,개인회생(폐지진행중),배정(대기),2021-03-15,2022-01-14,2020개회16387
4074,20410079,20106778,채무자,장광훈,폐지,절차폐지결정일,2024-10-17,진행,,,,개인회생(폐지진행중),배정(대기),2021-03-31,2021-10-20,2021개회3788


3.개시인데 왜 개시결정일 없음? 나우리 최신화 문제인듯
raw_data수정 : 현재결과일을 개시결정일에 추가해서 따로 할 건 없음


,채무자키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,현재결과일,분납상태,분납메모,중단사유,분납중단일,채무상태,담당자,개시결정일,변제계획인가일,사건번호
139,20527513,20158447,채무자,이정민,미인가,개시,2024-10-18,진행,,,,개인회생(진행중),개인회생,,,2024개회74477
149,20500143,20158448,채무자,이선희,미인가,개시,2024-10-21,진행,,,,개인회생(진행중),개인회생,,,2024개회221822
150,20500144,20158449,채무자,이선희,미인가,개시,2024-10-21,진행,,,,개인회생(진행중),개인회생,,,2024개회221822


In [ ]:
# ### 체크 후 수정
# raw_data.loc[[14],["인가미인가"]] = ["파기환송", "미인가"]

### 본격오류체크 - raw_data 변경 없으므로 생략가능. 이왕이면계좌 다운 전에 작업해서 수정반영
- 체크 순서 지켜야
- 오류관련 메시지는 분납비고에 작성

#### 1.신고

##### 1-1.개시신고(사건진행내용점검) : 채권자, 계좌신고여부 + 분납컷오프일>컷오프일

In [8]:
분납비고prefix = "개시신고_" # 항목별로 이것만 바꾸기
오류무시suffix = "오류(무시|아님)"
이달오류무시구문 = 분납비고prefix + cutoff[:4] + 오류무시suffix # 해당월에만 무시하기 위해 중간에 년월 삽입, 다른 항목에서도 동일한 변수명으로 통일했다.
항목오류무시구문 = 분납비고prefix + 오류무시suffix

### 진행건 중 채권자신고, 계좌신고 안 된 것 : 인가/미인가/not종결/메모는무시/개시결정
미종결개시건 = raw_data.query(               # str.contains('인가') : 미인가도 포함
                        '분납상태 == "진행" and 인가미인가.str.contains("인가") and 현재결과!="종결" \
                        and ~(상태및담당자.str.contains(@종료) or 상태및담당자.str.contains(@종결요청)) \
                        and 개시결정일 != ""'
                        )


미종결개시건[["채권자신고점검", "계좌신고점검", "분납컷오프점검"]] = "" # nan처리귀찮으니 ""로 초기화

### 타입변경s --------------------------------------------------
# 날짜는 날짜 타입으로
미종결개시건[['컷오프일','분납시작일', '분납컷오프일','변제계획인가일','신고조회일','변제시작일', '개시결정일', '계좌신고일']] = \
미종결개시건[['컷오프일','분납시작일', '분납컷오프일','변제계획인가일','신고조회일','변제시작일', '개시결정일', '계좌신고일']].apply(pd.to_datetime) # 날짜관련 에러나면 errors='coerce' 오류나면 이거 추가해라. 기본값은 raise

# 변제기일 변경
def convert_to_int(value):
    if pd.isna(value) or value == "":
        return pd.NA  # 빈 문자열은 NA로 반환
    if "일" in value:
        value = value.replace("일", "").strip()
        return int(value) if value.isdigit() else pd.NA
    return pd.NA

# '변제기일' 열에 사용자 정의 함수 적용
미종결개시건['변제기일'] = 미종결개시건['변제기일'].apply(convert_to_int)
### 타입변경e --------------------------------------------------


# 1) 채권자신고점검   # query는 정규식 표현 직접 사용 불가
채권자conds = ~미종결개시건.채권자목록.str.contains(검색용사명, regex=True)
미종결개시건.loc[채권자conds, "채권자신고점검"] = "채권자없음, "

# 2) 계좌신고점검
# 계좌신고서 송달일 도달일
송달검색어 = "\n(?P<발송일>\d{4}\.\d{2}\.\d{2}).*"+검색용사명+".*계좌(신고|신청)서\s?송달\x7f((?P<도달일>\d{4}\.\d{2}\.\d{2}))?"
def 계좌신고서(진행내용) : 
    검색결과 = re.search(송달검색어, 진행내용)
    if 검색결과 : 
        발송일 = 검색결과["발송일"] # 0은 전체내용
        도달일 = 검색결과["도달일"] # 2는 (솔림|헬프) 3은 (신고+신청)
        
        발송일 = datetime.strptime(발송일, "%Y.%m.%d")
        도달일 = datetime.strptime(검색결과["도달일"], "%Y.%m.%d") if 검색결과["도달일"] else pd.NaT
    else :
        발송일 = pd.NaT
        도달일 = pd.NaT
    return 발송일, 도달일

미종결개시건[["통지서발송일", "통지서도달일"]] = 미종결개시건["진행내용"].apply(lambda x : pd.Series(계좌신고서(x)))

# 개시결정후 반은 당일 또는 익일 발송됨. 일주일 이내 발송하는 것이 정상
# 최대 13일 이내에는 도달하고 있음(대부분 일주일 이내)
신고검색어 = 검색용사명 + ".*(계좌|채권).*(신고|신청).*제출"
신고안됨cond = ~미종결개시건["진행내용"].str.contains(신고검색어, regex=True) # 법원에 신고처리가 안 되어있음
신고처리안함cond = cutoff_dt - 미종결개시건['계좌신고일'] > pd.Timedelta(days=30) # 계좌신고일로 30일이 지남으나 반영안됨
신고안함cond = 미종결개시건['계좌신고일'].isna() & (cutoff_dt - 미종결개시건['통지서도달일'] > pd.Timedelta(days=30)) # 신고했다고 계좌신고일을 작성하지는 않으니, 일단 30일의 여유를 두고 오류체크
발송안함cond = 미종결개시건['통지서발송일'].isna() & (cutoff_dt-미종결개시건['개시결정일']>pd.Timedelta(days=7)) # 법원이 개시결정문과 신고서를 발송하지 않고있음
도달안됨cond = 미종결개시건['통지서도달일'].isna() & (cutoff_dt-미종결개시건['통지서발송일']>pd.Timedelta(days=14)) # 법원이 발송했으나 당사에 도달지연

미종결개시건.loc[신고안됨cond & 신고안함cond, '계좌신고점검'] += "계좌신고안함, "
미종결개시건.loc[신고안됨cond & 발송안함cond, "계좌신고점검"] += "통지서 발송이 7일 이상 지연 중, "
미종결개시건.loc[신고안됨cond & 도달안됨cond, "계좌신고점검"] += "통지서 도달이 14일 이상 지연 중, "
미종결개시건.loc[신고안됨cond & 신고처리안함cond, '계좌신고점검'] += "신고했으나 법원에 반영안됨, "
        

# 3) 분납컷오프일이 계좌컷오프일 이후인 경우 : 있을 수 없는일
분납컷오프오류cond = 미종결개시건.분납컷오프일>=미종결개시건.컷오프일
미종결개시건.loc[분납컷오프오류cond, "분납컷오프점검"] = "분납컷오프>=계좌컷오프, "

# 종합) 체크항목 합치기
미종결개시건["개시신고점검"] = 미종결개시건.채권자신고점검 + 미종결개시건.계좌신고점검 + 미종결개시건.분납컷오프점검
개시신고오류cond = 미종결개시건["개시신고점검"] != ""
개시신고오류무시cond = 미종결개시건.분납비고.str.contains(항목오류무시구문+'|'+이달오류무시구문, regex=True)


# 진행점검정상건이라 분납비고 삭제할 건 : 분납비고에 계좌신고, 채권자에 대한 내용이 있는 건
개시신고점검정상 = 미종결개시건[(~개시신고오류cond) & 개시신고오류무시cond][["분납키","채무자키","계좌키","분납비고"]]
개시신고점검정상.rename(columns={"분납비고":"구분납비고"},inplace=True)
개시신고점검정상["분납비고"] = "NULL"
개시신고점검정상 = 개시신고점검정상[["분납키","채무자키","계좌키","분납비고","구분납비고"]]


# 오류건 : 오류무시제외
개시신고점검오류 = 미종결개시건[개시신고오류cond & (~개시신고오류무시cond)]

##### 1-2. 미종결 인가건 정의(from 개시건)-------

In [55]:
# 인가신고오류무시 문구
분납비고prefix = "인가신고_"
이달오류무시구문 = 분납비고prefix + cutoff[:4] + 오류무시suffix # 해당월에만 무시하기 위해 중간에 년월 삽입, 다른 항목에서도 동일한 변수명으로 통일했다.
항목오류무시구문 = 분납비고prefix + 오류무시suffix

# 0. 대상한정 : 미종결개시건 중 정상건 이어받아서.
미종결인가건 = 미종결개시건[~개시신고오류cond].query('인가미인가=="인가" and ~(분납메모.str.contains("제외|일반|통합"))') 

# 정렬 : 관할법원,사건번호(주민번호는 차주번호임)
미종결인가건.sort_values(by=["관할법원","사건번호"], inplace=True)

# 1. 법원납입회차, 법원미납회차 작성
# pandas가 자동으로 ZeroDivisionError를 infinte로 처리해주므로 이렇게 써도 됐던것
미종결인가건["출금대상회차"] = (미종결인가건.출금대상잔액 / 미종결인가건.매월변제예정금액).round(3)

# 수정후) -------------------------------
# 1회차가 쌓여야 출금되는 걸 가정할 때, 
# 채무자 미납회차는 무조건 올림. 2.1회 미납이면 법원으로서는 3회 미입금 할 수 밖에.

# 출금대상회차
# 결론) 2회차 이상인 경우 체크 후 분납비고 작성할 것 EX) 당사출금대상(_입금요청완료_2408), 당사출금미대상
# 0회차~2회차 미만 : 정상 - 채무자가 납입일 넘겨서 입금하는 경우, 여기 쌓았다가, 채권자가 따로 입금요청하지 않으면 다음날에 입금됨
# 2회차 이상 : 비정상 - 채무자가 쓸데없이 돈을 먼저 냈거나, 계좌미신고 또는 별제권미확정 채권사의 공탁금이 여러회 쌓인경우. 당사가 여기에 포함되려면 개회입금내역이 아예 없거나 장기간 없었을 것
# 출금대상회차는 무조건 내림 = 입금요청가능한 회차수 = 법원입금처리 지연
미종결인가건["채미올림"] = np.ceil(미종결인가건.채무자미납회차)
미종결인가건["출대내림"] = np.floor(미종결인가건.출금대상회차)
미종결인가건["법원미납회차"] = 미종결인가건.채미올림 # + 미종결인가건.출대내림

# # 수정전) -------------------------------
# 기준소수 = 0.01
# 미종결인가건["법원미납회차"] = (미종결인가건.채무자미납회차 + 미종결인가건.출금대상회차).apply(lambda x: np.ceil(x) if (x - np.floor(x)) > 기준소수 else np.floor(x))
# ---------------------------------------

미종결인가건["법원납입회차"] = (미종결인가건.현재변제회차 - 미종결인가건.법원미납회차) # 출금대상잔액까지 감안함
미종결인가건["납입회차차이"] = (미종결인가건.납입회차 - 미종결인가건.법원납입회차) # 음수면 덜 들어온거
미종결인가건["현재회차차이"] = 미종결인가건.현재회차 - 미종결인가건.현재변제회차

# 2. 채권자변제시작일(실입금액 있는 회차 도래여부) to 변제현황조회서비스이용불가
def 채권자변제시작일찾기(row) :
    """인가일과 실입금액 있는 변제일 비교해서"""
    for i in range(1, 11):
        if row[f'변제예정액{i}'] > 0 :
            채권자변제시작일 = datetime.strptime(row[f'변제기간{i}'][:10], "%Y-%m-%d")
            채권자변제시작회차 = int(row[f'변제회차{i}'].split("~")[0])
            if 채권자변제시작일 <= row.변제계획인가일 :
                채권자변제시작일 = 특정날짜후첫변제일(row.변제계획인가일, row.변제기일)
            return 채권자변제시작일, 채권자변제시작회차
        
미종결인가건[["채권자변제시작일", "채권자변제시작회차"]] = 미종결인가건.apply(lambda row : pd.Series(채권자변제시작일찾기(row)), axis=1)


# 3. 초회여부, 당사변제시작일(실입금) : 당사시작회차 == 채권자시작회차
# 입금컷오프일 : 초회여부 함수에 필요함
미종결인가건["입금컷오프일"] = 미종결인가건[["컷오프일", "분납컷오프일"]].max(axis=1)
def 초회여부(row) :
    if pd.isna(row.변제계획인가일) or pd.isna(row.분납시작일) or pd.isna(row.입금컷오프일) or pd.isna(row.변제기일):
        return pd.NaT, "변제오류"
    
    인가일, 분납시작일, 입금컷오프일, 변제기일 = row.변제계획인가일, row.분납시작일, row.입금컷오프일, row.변제기일
    
    # 인가 + 변제기일
    인가후첫변제일 = 특정날짜후첫변제일(인가일, 변제기일)
    
    # 컷오프 + 변제기일 : 변제일과 컷오프의 날짜를 비교하여 컷오프 후 첫 날짜 
    컷오프후첫변제일 = 특정날짜후첫변제일(입금컷오프일, 변제기일)
        
    # 법원 첫 입금예정일
    # 채권자 변경하면서 변경이후회차부터 변제현황을 입력하는 경우도 있어서(제주지방법원, 강릉지원?, 의정부-일부)
    # 처음에는 채권자변제시작일을 여기서는 제외했었는데, 그런 경우는 굉장히 드물어서 다시 포함함. 전후 초회리스트 비교하여 중간으로 변경할 건 확인 완료
    법원첫입금일 = max(row.채권자변제시작일, 인가후첫변제일, 분납시작일) 
    
    # 당사변제시작일
    당사변제시작일 = max(row.채권자변제시작일, 컷오프후첫변제일, 인가후첫변제일)
    
    if 법원첫입금일 >= 컷오프후첫변제일 :
        return 당사변제시작일, "초회"
    else :
        return 당사변제시작일, "중간"

미종결인가건[["당사변제시작일", "초회여부"]] = 미종결인가건.apply(lambda row : pd.Series(초회여부(row)), axis=1)

#### -------------------------------------------------------------------------------------
# 실제로는 중간회차인 건 : 채권자변제시작일상 초회부터이나 법원에서 이전 매각사 상환스케줄을 잘라버린 경우
# 재접수하면 분납키가 바뀌니까 수정할 필요는 없다. 다만 완납한 경우는 리스트에서 삭제해주자
중간회차분납키 = ["20135963", "20136732", "20135994"]
초회여부변경조건 = 미종결인가건["분납키"].isin(중간회차분납키)
미종결인가건.loc[초회여부변경조건,"초회여부"] = "중간"


# 수정전 : 채권자변제시작일을 법원첫입금일에 포함시키지 않았던 때
# 초회분납키 = ["20105064", "20105488"]
# 초회여부변경조건 = 미종결인가건["분납키"].isin(초회분납키)
# 미종결인가건.loc[초회여부변경조건,"초회여부"] = "초회"

print('완납중단인 경우 초회분납키리스트에서 삭제, 97번줄')
초회건중중단건index = 미종결인가건.loc[초회여부변경조건 & (미종결인가건["분납상태"] == "중단")].index
display(raw_data.loc[초회건중중단건index, 기본조회칼럼])
#### -------------------------------------------------------------------------------------


# 3. 최근변제예정일찾기 : 컷오프일 전후로 가장 가까운 변제일, 컷오프일이나 조회일이나 실질적인 차이는 없다.
def 최근변제예정일찾기(변제기일, 컷오프일:datetime):
    if pd.isna(변제기일) : return pd.NaT
    
    조회년도 = 컷오프일.year
    조회월 = 컷오프일.month
    
    # 참조일 : 현재월의 1일(1일은 어느 달이나 있으니까)
    참조일 = datetime(조회년도, 조회월, 1)
    참조일이전달 = 참조일 - relativedelta(months=1)
    참조일다음달 = 참조일 + relativedelta(months=1)
    
    # 이번달, 지난달, 다음달 변제일 계산
    이번달변제일 = 말일오류없애기(참조일, 변제기일)
    이전달변제일 = 말일오류없애기(참조일이전달, 변제기일)
    다음달변제일 = 말일오류없애기(참조일다음달, 변제기일)
  
    
    # 조회일과 가장 가까운 날짜 계산
    최근변제예정일 = min([이번달변제일, 이전달변제일, 다음달변제일], 
                       key=lambda date: abs(컷오프일 - date))
    
    return 최근변제예정일

미종결인가건["최근변제예정일"] = 미종결인가건.apply(lambda row : 최근변제예정일찾기(row.변제기일, cutoff_dt), axis=1)

# 현재회차 변제금(분납컷오프일 계산시 사용) : 0이면 아직 시작안 했으므로 분납컷오프일을 수정할 일이 없어야 한다. 0일지라도은 0대로 두고 본다.
def 현재회차변제금찾기(row) : # 단순하게 현재회차로 한다.(납입회차는 점검대상이고, 중간회차부터 시작하는 경우때문에 복잡)
    for i in range(1,11) :
        구간마지막회차 = row[f'변제회차{i}'].split("~")[-1]
        
        if 구간마지막회차 == "" : # 변제오류건
            return pd.NA
        
        구간마지막회차 = int(구간마지막회차)
        if row["현재회차"] <= 구간마지막회차 : # 낮은 회차부터 순회하므로 처음 걸리는 곳이 현재회차가 속한 구간이다.
            현재회차변제금 = int(row[f'변제예정액{i}'])
            return 현재회차변제금
    
    # 현재회차 변제금을 못찾은 경우 에러처리
    return pd.NA

미종결인가건["현재회차변제금"] = 미종결인가건.apply(lambda row : 현재회차변제금찾기(row), axis=1) # pd.Series로 감싸는 건 반환값이 2개 이상일 때

# 마지막회차변제금찾기 : 변제금 증감 확인용
def 마지막회차변제금찾기(row) : 
    마지막회차변제금 = pd.NA
    for i in range(10, 0, -1) :
        변제회차값 = str(row[f'변제회차{i}']) if not pd.isna(row[f'변제회차{i}']) else ''
        if (row[f'변제예정액{i}'] > 0) and re.search('-', 변제회차값) is None:
            마지막회차변제금 = row[f'변제예정액{i}']
            break
    return 마지막회차변제금

미종결인가건["마지막회차변제금"] = 미종결인가건.apply(lambda row : 마지막회차변제금찾기(row), axis=1) 

# 일시변제금액, 재산처분금액 찾기       
def 일시재산변제금찾기(row) : 
    일시변제금액, 재산처분금액 = 0, 0
    for i in range(10, 0, -1) :
        if row[f'변제회차{i}'] == "-1" :
            일시변제금액 += row[f'변제예정액{i}'] # 혹시 여러개일 수도
        elif row[f'변제회차{i}'] == "-2" :
            재산처분금액 += row[f'변제예정액{i}'] # 혹시 여러개일 수도
    return 일시변제금액, 재산처분금액

미종결인가건[["일시변제금액", "재산처분금액"]] = 미종결인가건.apply(lambda row : pd.Series(일시재산변제금찾기(row)), axis=1) 


# OPB와 총분납금 비교칼럼 작성
미종결인가건["OPB-총분납금"] = 미종결인가건["OPB"] - 미종결인가건["총분납금"]

완납중단인 경우 초회분납키리스트에서 삭제, 97번줄


,채무자키,계좌키,분납키,분납자관계,분납자성명인,주민번호인,관할법원,사건번호,인가미인가,현재결과,분납메모,분납기타,분납비고,기타금액


##### 1-3.인가신고점검 : (인가) 필수입력값

In [10]:
미종결인가건[["제외점검", "채권번호점검", "신고조회결과점검"]] = ""

# 1) 제외건류(0원짜리)
제외conds = (미종결인가건.총분납금==0) & ((미종결인가건.최초원금==0)|(미종결인가건.현재원금==0))
미종결인가건.loc[제외conds, "제외점검"] = "제외확인, "


# 2) 채권번호 오류 : 
    # 첫번째 조건: 변제채권번호스캔과 변제채권번호열이 빈값인 경우는 조회 실패라서 신고조회결과 체크에서 걸러짐
    # 따라서 변제채권번호스캔과 변제채권번호 둘 중 하나라도 빈값이 아닌 경우(변제조회 성공,채권자정보없더라도)를 대상으로 점검한다.
기본cond = (미종결인가건['변제채권번호스캔'] != "") | (미종결인가건['변제채권번호'] != "")
    # 두번째 조건 : 변제현황 조회 성공인데, 입력값과 실제값이 다른 경우
채권번호cond2 = 미종결인가건.변제채권번호입력 != 미종결인가건.변제채권번호
    # 세번째 조건: 변제채권번호스캔이 변제채권번호를 포함하지 않는 경우
채권번호cond3 = ~미종결인가건.apply(lambda row: row['변제채권번호'] in row['변제채권번호스캔'], axis=1)
    # 각 조건을 합쳐서 불리언인덱싱
채권번호conds = 기본cond & (채권번호cond2|채권번호cond3)
미종결인가건.loc[채권번호conds, "채권번호점검"] = "채권번호확인, "

# 3) 신고조회결과(변제조회 이용가능) : 계좌정보오등록은 3)과 중복될 수 있음
조회결과conds = [
    미종결인가건.신고조회결과.str.match('은행코드') & (cutoff_dt - 미종결인가건.변제계획인가일 > pd.Timedelta(days=30)), # 인가후 한달 지났는데 변제조회 오류
    미종결인가건.신고조회결과.str.match('채권자') & (cutoff_dt - 미종결인가건.변제계획인가일 > pd.Timedelta(days=30)), # 인가후 한달 지났는데 변제조회는 되나 당사 정보 없음
    
    # 변제현황조회 서비스 이용불가건 : 실입금회차 시작여부가 중요. 채권자든 당사든 실입금 기준이라 변제조회오류결과는 동일하게 나옴
    # 미종결인가건.변제현황조회.str.match('변제현황조회')&(미종결인가건.채권자변제시작일>=cutoff_dt), # 변제일미도래(정상)
    # 미종결인가건.변제현황조회.str.match('변제현황조회')&(미종결인가건.채권자변제시작일<cutoff_dt), # 변제일도래(비정상-입금불가사유 파악 및 입금요청 필요->변제내용에서 함)
    
    # 필수항목오류 : 위 두 조건에서 이미 걸러질 것이나 혹시 몰라 한 번 더
    미종결인가건.관할법원 == "",
    미종결인가건.사건번호 == "",
    (미종결인가건.신고은행 == "")|((미종결인가건.입금은행!="")&(미종결인가건.신고은행!=미종결인가건.입금은행)), # 입금은행이 빈값인 것은 첨부터 변제조회 실패건
    (미종결인가건.신고계좌 == "")|((미종결인가건.입금은행!="")&(미종결인가건.신고계좌!=미종결인가건.입금계좌)),
    (~미종결인가건.신고채권자.str.match(신고채권자사명))|((미종결인가건.입금은행!="")&(미종결인가건.신고채권자!=미종결인가건.입금채권자)),
    (미종결인가건.입금은행=="") & (미종결인가건.총분납금==0) # 막 인가되어서 변제현황 안 읽어지는 경우
    ]

조회결과values = [
    "계좌정보오등록(인가후30일초과), ",
    "채권자현황오류(인가후30일초과), ",

    # "이용불가_변제일미도래",
    # "이용불가_변제일도래",
    
    "관할법원없음, ",
    "사건번호없음, ",
    "신고은행오류, ",
    "신고계좌오류, ",
    "신고채권자오류, ",
    "총분납금0원, "
    ]
미종결인가건['신고조회결과점검'] = np.select(조회결과conds, 조회결과values, default="")

# 종합
미종결인가건["인가신고점검"] = 미종결인가건.제외점검 + 미종결인가건.채권번호점검 + 미종결인가건.신고조회결과점검

인가신고오류cond = 미종결인가건["인가신고점검"] != ""
인가신고오류무시cond = 미종결인가건.분납비고.str.contains(항목오류무시구문+'|'+이달오류무시구문, regex=True)


# 정상건이라 분납비고 삭제할 건 
인가신고점검정상 = 미종결인가건[(~인가신고오류cond) & 인가신고오류무시cond][["분납키","채무자키","계좌키","분납비고"]]
인가신고점검정상.rename(columns={"분납비고":"구분납비고"},inplace=True)
인가신고점검정상["분납비고"] = "NULL"
인가신고점검정상 = 인가신고점검정상[["분납키","채무자키","계좌키","분납비고","구분납비고"]]

# 오류건
인가신고점검오류 = 미종결인가건[인가신고오류cond & (~인가신고오류무시cond)]

# 정렬
인가신고점검오류.sort_values(by=['인가신고점검','관할법원','사건번호'], inplace=True) 

##### 출력 : 신고오류 및 미종결인가건

In [40]:
# 출력
    # 오류건 : 개시신고와 인가신고 통합
개시신고오류cols = 기본조회칼럼 +["개시신고점검", "개시결정일", "통지서발송일", "통지서도달일","계좌신고일", "변제계획인가일", "총분납금","총분납잔금"] + 변제조회칼럼
인가신고오류cols = 기본조회칼럼 +["인가신고점검", "개시결정일", "통지서발송일", "통지서도달일","계좌신고일", "변제계획인가일", "총분납금","총분납잔금"] + 변제조회칼럼
통합cols = 기본조회칼럼 +["개시신고점검","인가신고점검", "개시결정일", "통지서발송일", "통지서도달일","계좌신고일", "변제계획인가일", "총분납금","총분납잔금"] + 변제조회칼럼

신고오류 = pd.concat([개시신고점검오류[개시신고오류cols], 인가신고점검오류[인가신고오류cols]])[통합cols]
신고오류 = 신고오류.sort_values(by=["관할법원", "사건번호"])


if len(신고오류) > 0 : 
    print(f'개시신고점검 오류 : {len(개시신고점검오류)}건. 엑셀출력')
    print(f'인가신고점검 오류 : {len(인가신고점검오류)}건. 엑셀출력')
    functions.save_df_to_excel_underline(신고오류,join(path_wd_check, "[check]개인회생_1.신고점검오류_"+cutoff+".xlsx"), key_columns_no=6)
else : 
    print('신고 점검 이상무')
    


    # 정상건 : 분납비고 삭제건
신고정상 = pd.concat([개시신고점검정상, 인가신고점검정상])

if len(신고정상) > 0 : 
    print(f'분납비고 삭제 체크 : {len(개시신고점검정상)}건. 엑셀출력')
    print(f'분납비고 삭제 체크 : {len(인가신고점검정상)}건. 엑셀출력')
    functions.save_df_to_excel_underline(신고정상, join(path_wd_check, "[전산수정]개인회생_1.신고점검정상_분납비고삭제_"+cutoff+".xlsx"), key_columns_no=0)
else : 
    print('분납비고 삭제할 건 없음')

개시신고점검 오류 : 2건. 엑셀출력
인가신고점검 오류 : 3건. 엑셀출력
분납비고 삭제할 건 없음


In [57]:
# 미종결인가건 출력 : 참고용으로 볼거면 출력(변제조회 최신화 한 뒤에)
미종결인가건cols = 기본조회칼럼 + 변제조회칼럼 + 전산회차칼럼 + 법원회차칼럼 + 회차보조칼럼 + 변제분석칼럼 + 보조칼럼 + ["OPB-총분납금","최초OPB","최초원금", "초과입금", "현재회차변제금", "마지막회차변제금", "일시변제금액", "재산처분금액"]

# # 납입 종료건
# 미종결인가건.query()
functions.save_df_to_excel_underline(미종결인가건[미종결인가건cols], join(path_wd_check, "[참고용]개인회생_미종결인가건_"+cutoff+".xlsx"), key_columns_no=6)

#### 2. 최신화 및 동일사건 회차 확인(1,2pass가능)
- 2번까지 처리 후 전산 재다운로드

##### 2-1.변제최신화실패 : 변제최신화일 < 컷오프일

In [41]:
# 아침에 정상적으로 최신화 돌았으면 전날(컷오프) 법원 업무종료시까지의 데이터가 업데이트된다.
# 따라서 신고조회일이 컷오프일보다 작은 경우만 최신화 후보
# 그중 현재회차변제예정일이 컷오프일 근처인 것은 최신화가 반드시 필요(250개 하는데 15분 소요 > 필요한 것만 하자)
# 컷오프는 조회 전날이므로 신고조회일이 컷오프일보다 작으면서 변제예정일이 3일 이내인 것만 하자.

# 변제예정일과 조회일의 차이가 3일 이내
미종결인가건['날짜차이'] = (미종결인가건['최근변제예정일'] - 미종결인가건['신고조회일']).abs().dt.days

# 변제조회 최신화 필요건
최신화cond1 = 미종결인가건["신고조회일"] < cutoff_dt # 컷오프 실패건 or 주말껴서 컷오프 안 된 건(주말이라도 법원회차계산 되어야지)
최신화cond2 = 미종결인가건['날짜차이'] <= 3 # 컷오프 == 조회일 일때
최신화cond3 = ~((미종결인가건['현재회차차이'] != 0) & (미종결인가건.초회여부=="초회")) # 실변제 첫회부터 시작하면서 현재회차차이가 0이면 최신화 안 해도 됨
# 변제현황조회서비스불가건이라도 미납회차와 출금예정금액 변화가 있을 수 있으므로 최신화 필요함
# 최신화cond3 = ~미종결인가건["변제현황조회"].str.match("변제현황조회") # 변제현황조회서비스이용불가건은 입금없는 건이므로 법원회차 최신화의 의미가 없다.(변제중이라도 모두 미납)

# 컷오프일과 조회일이 같으면 날짜차이 상관없이 모두 최신화 대상(현재회차 등 업데이트 해야 하므로)
# 그러나 중간에 하면 당연히 오차가 발생하게 되므로 법원업무 끝나고 하거나, 전날을 기준으로 해야 한다.
if cutoff_dt == 조회일_dt : 
    최신화conds = 최신화cond1 & 최신화cond2
# 이 경우가 일반적인 경우 : 현재회차,미납회차 등에 변화가 있을 수 있는 것만 골라서 최소한으로
else : 
    최신화conds = 최신화cond1 & 최신화cond2 & 최신화cond3

# 변제조회최신화대상 = 미종결인가건.query("신고조회일 != @조회일")
변제조회최신화대상 = 미종결인가건.loc[최신화conds]

변제조회최신화대상['신고조회일'] = 변제조회최신화대상['신고조회일'].dt.strftime('%Y-%m-%d')

# # 사건조회 파일 만들기 : 사건조회는 최신화 내용이 개인회생상세에 반영되지 않는다.
# # 그냥 변제최신화 버튼 직접 누른다.
# 사건조회cols = ["사건키", "관할법원", "사건번호", "당사자", "채권자", "신고은행", "신고계좌", "신고채권자"]
# 사건조회 = pd.DataFrame(None, columns=사건조회cols)
# 사건조회["관할법원"] = 변제조회최신화대상.관할법원
# 사건조회["사건번호"] = 변제조회최신화대상.사건번호
# 사건조회["당사자"] = 변제조회최신화대상.분납자성명인
# 사건조회["채권자"] = "헬프"
# 사건조회["신고은행"] = 변제조회최신화대상.신고은행
# 사건조회["신고계좌"] = 변제조회최신화대상.신고계좌
# 사건조회["신고채권자"] = "헬프"

# # 중복제거
# 사건조회.drop_duplicates(["관할법원", "사건번호"], inplace=True)

# 엑셀저장
# save_df_to_excel(사건조회, join(path_wd, "[최신화요청]개인회생사건조회_"+조회일+".xlsx"))
변제조회최신화대상cols = ["채무자키", "분납키", "분납자성명인", "관할법원", "사건번호", "신고조회일", "변제기일", "현재회차차이", "현재회차", "현재변제회차", "당사변제시작일", "채권자변제시작일","변제시작일"]

if len(변제조회최신화대상) > 0 :
    print("유의사항 : 동일한 사건도 분납키마다 최신화 필요. 현재회차차이가 발생한 건만 해도 될듯 >> 확인되면 이렇게 수정")
    functions.save_df_to_excel_underline(변제조회최신화대상[변제조회최신화대상cols], join(path_wd_check, "[check]개인회생_2-1.최신화요청_사건조회_"+조회일+".xlsx"), key_columns_no=5)
else :
    print('최신화 할 건 없음')

### 일괄 최신화 후에는 변제오류가 있더라도 신고조회일을 바뀐다.
### 따라서 신고조회일로 오류 따지는 것은 더이상 의미없고, 신고결과 등을 봐야한다.

최신화 할 건 없음


##### 2-2.동일사건,다른 회차 - 현재회차2

In [43]:
# '관할법원+사건번호'로 그룹화 : 주민번호는 분납자가 아니라 채무자라서 정확하지 않음
# '현재회차'고유값이 둘 이상인 건
# '주민번호인'으로 그룹화하고 '현재회차'의 고유값이 2개 이상인 그룹 필터링
동일사건다른회차 = 미종결인가건.groupby(by=["관할법원", "사건번호"]).filter(lambda x: x['현재회차'].nunique() >= 2 or x['미납회차'].nunique() >= 2)
#############################################
# 한번씩 실사 후 상황바뀌면 수정할 것
if company == "솔림" : 
    동일사건회차차이무시 = ("20131475", "20131314",  # 별제권미확정신고건 최규송
                  "20131202", "20131473" # 20515398 여기에 미입금 3회분 곧 입금해준다고. 10월, 11월 자산정리 전에 필히 확인 ####################
                  )
                  
    동일사건다른회차 = 동일사건다른회차[~동일사건다른회차["분납키"].isin(동일사건회차차이무시)] 
else : 
    pass
#############################################

동일사건다른회차cols = 기본조회칼럼 +["초과입금", "현재회차변제금"]+ 회차보조칼럼 + 전산회차칼럼 + 법원회차칼럼 + 변제분석칼럼

# 정렬
동일사건다른회차.sort_values(by=["관할법원","사건번호","채무자키"], inplace=True)

if len(동일사건다른회차) : 
    print(f"동일사건에 현재회차가 다른 경우 {len(동일사건다른회차)}건. 엑셀 확인")
    print('수정후 분납 새로고침 필수!!--------------------')
    functions.save_df_to_excel_underline(동일사건다른회차[동일사건다른회차cols], join(path_wd_check, "[check]개인회생_2-2.동일사건다른회차_"+cutoff+".xlsx"), key_columns_no=6)
else :
    print("이상무")


이상무


##### 2-3.동일사건 채권번호 혼동건, 변제정상건 정의

In [11]:
분납비고prefix = "변제내용_"
이달오류무시구문 = 분납비고prefix + cutoff[:4] + 오류무시suffix # 해당월에만 무시하기 위해 중간에 년월 삽입
항목오류무시구문 = 분납비고prefix + 오류무시suffix 

# 변제정상건 정의 및 한정 : 변제조회정상, 채권자변제일도래(법원입금이 한 번도 되지 않아서 변제조회불가건이라도, 조회성공했고, 변제일 도래했다면 당연 검수대상)
변제성공조건 = 미종결인가건["신고조회결과"]=="" # 인가신고점검 등은 반영하지 않는다.
변제일도래조건 = (cutoff_dt - 미종결인가건["채권자변제시작일"]).dt.days >= 30 # 변제시작일로부터 30일 이상된 것
변제정상건 = 미종결인가건[변제성공조건 & 변제일도래조건]

# 납입회차 0 인경우, 채권자변제시작회차가 1회차가 아닌경우 당사시작회차 시작전이면 오류 점검 하지 않기 위해 전산 납입회차 정정
    # EX) 당사납입회차 0, 시작회차 15회, 현재회차및법원납입회차 15이하라면 오류검사 필요없다. 
    # 오류 검수는 납입회차 차이가 0보다 작은 경우를 대상으로 하기때문에, 변제시작회차까지는 납입됐다고 처리해도 문제없음
    # 당사 납입이 이루어진 경우는 그대로 두어야!
    # 중간회차에도 적용해도 됨을 확인함
납입회차수정조건 = (변제정상건["납입회차"]==0) & (변제정상건["채권자변제시작회차"] > 1)
변제정상건["납입회차"] = np.where(납입회차수정조건, 변제정상건["채권자변제시작회차"]-1, 변제정상건["납입회차"])
    # 납입회차차이 재작성
변제정상건["납입회차차이"] = 변제정상건["납입회차"] - 변제정상건["법원납입회차"]


#### 동일사건 변제채권번호 확인 ----------------------------------------------------
채권번호혼동 = 변제정상건[변제정상건.duplicated(["관할법원", "사건번호"], keep=False)]

# 그룹별 순위 구하기 : OPB와 최초원금간 순위가 다른 경우까지 살펴보자. 오등록 확인위해
채권번호혼동[['최초OPB_순위',"매입OPB_순위",'총분납금_순위']] = 채권번호혼동.groupby(['관할법원', '사건번호'])[['최초OPB','OPB','총분납금']].transform(lambda x: x.rank(ascending=False))


# 셋중에 하나라도 다르면 확인
채권번호혼동 = 채권번호혼동[(채권번호혼동['최초OPB_순위'] != 채권번호혼동['매입OPB_순위']) | (채권번호혼동['매입OPB_순위'] != 채권번호혼동['총분납금_순위'])]

# 오류아님을 확인했거나 금액이 비슷하여 오류무시해도 되는 건 삭제
채권번호혼동 = 채권번호혼동[채권번호혼동["분납비고"] != 채권번호혼동["변제채권번호입력"]]

# 그룹별로 보기 위해 정렬
채권번호혼동 = 채권번호혼동.sort_values(by=['관할법원', '사건번호', '채무자키'])      

순위다른채권cols = 기본조회칼럼 + ["최초OPB_순위","매입OPB_순위","총분납금_순위"] + 채권번호칼럼 + ["매각사구분","채권구분","최초원금","최초법비용", "OPB", "총분납금"]

if len(채권번호혼동) > 0 :
    functions.save_df_to_excel_underline(채권번호혼동[순위다른채권cols], join(path_wd_check, "[check]개인회생_2-3.채권번호혼동확인_"+조회일+".xlsx"), key_columns_no=6)
else :
    print('채권번호 혼동건 없음(금액순위 다른 채권 없음)')
# ---------------------------------------------------------------------------

채권번호 혼동건 없음(금액순위 다른 채권 없음)


#### 3. 변제내용점검 : 회차, 금액
- 점검은 자산정리 후 

##### 3-1.총분납금 vs OPB (중간회차)

In [12]:
# 세부항목 오류무시 구문
총분납금오류무시구문 = 분납비고prefix + "총분납금" + 오류무시suffix # opb-총분납금 > 0 이나 맞출수가 없고, 무시해도 되는 건

중간회차 = 변제정상건.query('초회여부=="중간"')

# OPB가 분납잔금으로 입력되었는지, 솔림은 위에서 수정했기 때문에 더이상 확인할 필요없음 -------------------
중간회차["OPB가인가잔액"] = np.where(((중간회차["최초OPB"]==중간회차["OPB"]) | (중간회차["현재원금"]==중간회차["OPB"])), "원금잔액", "인가잔액")

# 정상건 제외 : opb와도 같고, 회차차이도 없는 건은 삭제
# 현재회차 1회차, 신고조회일과 조회일일 1차이인 경우는 수가 적으니 직접 확인한다.
중간회차 = 중간회차[~((중간회차["OPB-총분납금"]==0) & (중간회차["납입회차차이"]==0) & (중간회차["현재회차차이"]==0))] 

# OPB가 인가잔액으로 모두 수정된 것을 가정하고 총분납금이 OPB보다 작은 경우에 대해
def 분납컷오프일수정하기(row):
    현재당사시작회차 = pd.NA
    수정된당사시작회차 = pd.NA
    수정된분납컷오프일 = pd.NA
    if row["OPB-총분납금"] > 0:
        # 스케줄 작성
        schedule = []
        for i in range(1, 11):
            if row[f'변제회차{i}'] in ["", '-1', '-2'] or row[f'변제예정액{i}'] == 0:
                continue
            else:
                s = int(row[f'변제회차{i}'].split('~')[0])
                e = int(row[f'변제회차{i}'].split('~')[-1])
                for j in range(s, e + 1):
                    schedule.append((j, row[f'변제예정액{i}']))

        schedule_sr = pd.Series(dict(schedule))

        # 당사변제시작회차(현재입력된 컷오프일기준)
        실컷오프일 = row.컷오프일 if pd.isna(row.분납컷오프일) else row.분납컷오프일
        
        스케줄시작일 = 특정날짜후첫변제일(실컷오프일, row["변제기일"])
        
        diff = relativedelta(스케줄시작일, row['변제시작일'])
        현재당사시작회차 = diff.years*12 + diff.months + 1
        
            # 변제기간1이 컷오프보다 훨씬 나중인 경우
        if 현재당사시작회차 < row.채권자변제시작회차 : 
            현재당사시작회차 = row.채권자변제시작회차
        
        # OPB값과 비교
        total = 0
        for i in reversed(schedule_sr.index):
            total += schedule_sr[i]
            if total == row["OPB"]:
                수정된당사시작회차 = i
                수정된분납컷오프일 = row["변제시작일"] + pd.DateOffset(months=(i-1)) - pd.DateOffset(days=1) # 1회차가 변제시작일이므로 (i-1), 하루빼줘야 그 날이 스케줄에 포함됨
                return pd.Series([수정된당사시작회차, 수정된분납컷오프일, 현재당사시작회차])
            elif total > row["OPB"]:
                if row["OPB"] == (total - schedule_sr[i]) + schedule_sr[schedule_sr.index[0]]:
                    수정된당사시작회차 = str(i + 1) + ", 초회"  # reversed이므로 +1 임에 유의
                    수정된분납컷오프일 = row["변제시작일"] + pd.DateOffset(months=i) - pd.DateOffset(days=1)
                    return pd.Series([수정된당사시작회차, 수정된분납컷오프일, 현재당사시작회차])
                else:
                    수정된당사시작회차 = "DD확인"
                    return pd.Series([수정된당사시작회차, 수정된분납컷오프일, 현재당사시작회차])
            else:
                continue
        # 초회까지 다 더해도 opb를 넘지 못하는 경우
        수정된당사시작회차 = "OPB달성불가"
        return pd.Series([수정된당사시작회차, 수정된분납컷오프일, 현재당사시작회차])
    return pd.Series([수정된당사시작회차, 수정된분납컷오프일, 현재당사시작회차])


중간회차[["수정된당사시작회차", "수정된분납컷오프일","현재당사시작회차"]] = 중간회차.apply(lambda row: 분납컷오프일수정하기(row), axis=1)

In [ ]:
# 전달미인가종결건과 이번달 미인가종결건 또는 중간회차 비교를 통해
# 전달과 변제금 달라진 거
# -회차 추가/변동 여부
# -회차 입금여부
# 초회 입금여부

##### 3-2.회차 및 초과입금 (전체)

In [13]:
# 변제정상건 재정의 하는 코드 없으니 여기만 반복실행해도 됨
if True :
    # 점검항목별 오류무시구문
        # 초과입금 발생원인 중 미등록계좌,매입전회차상승분,-회차,오입금은 '회생폐지'를 통해 애초에 발생하지 않게 처리한다.
        # 따라서 초과입금은 선입금의 경우만 존재가능, 선입금이 확인되면 비고에 오류무시구문 작성
    변제내용점검항목 = ["총분납회차", "현재회차", "납입회차", "초과입금"]
    변제내용오류무시구문 = []
    변제내용오류구문 = []
    for 항목명 in 변제내용점검항목 : 
        변제내용오류무시구문.append(분납비고prefix + 항목명 + 오류무시suffix)
        # 오류구문
        if 항목명 == "납입회차" : 
            변제내용오류구문.append(항목명 + "부족") # 납입회차오류values에서 ', '는 추가
        else : 
            변제내용오류구문.append(항목명 + "체크, ")

    # 오류무시건은 2-3에서 제거함
    # ((변제정상건["최초OPB-총분납금"]<0)&(~변제정상건["분납비고"].str.contains("회차점검_OPB초과변제", na=False))), #개인회생은 원리금 인정되는 경우 있어 초회중 OPB보다 큰 건 문제가 안 됨.
    # 1. 총분납회차
    총분납회차오류cond = (변제정상건.총분납회차!=변제정상건.전체변제회차)
    # 2. 현재회차차이오류 : 차이가 있는 건 중 +1회차 차이는 예정일이 조회일과 같으면 오류무시
    현재회차오류cond = ((변제정상건["현재회차차이"]==1)&(변제정상건["최근변제예정일"]!=조회일_dt))|((변제정상건["현재회차차이"].abs()>1)|(변제정상건["현재회차차이"]==-1))
    # 3. 납입회차차이오류 : 미납소수부분과 출금대상회차로 정확히 커버가 안 되는 경우
        # 초과입금(선입금) 아닌 이상 많이 들어오는 건 상관없으니 납입회차차이가 양수인것은(전산>법원) 오류제외
        # 현재회차가 당사납입회차보다 작은 경우는 납입회차차이가 0보다 크도록 만들어두었다.. 
    납입회차오류cond = [                                                                                                                         # 나머지
        (변제정상건["납입회차차이"]<0) & (변제정상건["초회여부"]=="초회") & (변제정상건["납입회차차이"].abs() > np.ceil(np.modf(변제정상건["채무자미납회차"])[0] + 변제정상건["출금대상회차"])),
        (변제정상건["납입회차차이"]<0) & (변제정상건["초회여부"]!="초회") & (변제정상건["납입회차차이"].abs() > np.ceil(np.modf(변제정상건["채무자미납회차"])[0] + 변제정상건["출금대상회차"])),
        # 출금대상회차로 커버가 되더라도 출금대상회차가 5회가 넘어가는 경우 
        # 납입이 시작된 이상 납입회차차이는 양수일수는 있어도 음수이면 안 됨. 아래 조건으로 커버됨
        (변제정상건["납입회차차이"]<=-5) & (변제정상건["출금대상회차"]>4)]

    납입회차오류values = [
        변제내용오류구문[2] + "_공탁금확인(당사만입금덜됨), ",
        변제내용오류구문[2] + "_인아웃터림or법원입금요청(매입건), ",
        변제내용오류구문[2] + "_법원입금요청(출금대상잔액5회이상쌓임), "
        ]

    # 3. 초과입금
    초과입금오류cond = [
        변제정상건["초과입금"] > 0, 
        변제정상건["총분납금"]<변제정상건["총분납입금"], # 총분납입금이 더 많다면 회차무관, 초회여부 무관 점검대상
        # 총분납금=총분납입금 같더라도 법원미납회차가 1이상이라면 점검대상(출금대상잔액은 보지않는다. 다른 채권사 문제) 
        ###########################################
        # 중간회차는 채권자변경기간에 공탁이 발생할 수 있고, 매각사가 계산을 잘못했다면 받을 게 있을 수 있으니 일단 테스트로 중간회차 한두개 전화확인해보자. 출금대상잔액 뭐냐고
        (변제정상건["총분납금"]==변제정상건["총분납입금"]) & (변제정상건["법원미납회차"]>=1) 
        ]
    초과입금오류values = [
        변제내용오류구문[3] +", ",
        변제내용오류구문[3] + "_총분납금초과, ",
        변제내용오류구문[3] + "_법원잔여회차있음"
    ]


    # 각각 점검 칼럼에 작상
    변제정상건["총분납회차점검"] = np.where(총분납회차오류cond,변제내용오류구문[0],"")
    변제정상건["현재회차점검"] = np.where(현재회차오류cond,변제내용오류구문[1],"")
    변제정상건["납입회차점검"] = np.select(납입회차오류cond,납입회차오류values,default="")
    변제정상건["초과입금점검"] = np.select(초과입금오류cond,초과입금오류values,default="")

# 총분납금점검결과 합하기
if True : 
    # +) 총분납금 오류 검사결과 불러오기
    총분납금오류cond = 중간회차["OPB-총분납금"]>0
    총분납금오류ind = 중간회차[총분납금오류cond].index
    변제정상건["총분납금점검"] = "" # 초기화 필수!! (na와 ""의 차이때문에)
    변제정상건.loc[총분납금오류ind, "총분납금점검"] = "매입OPB과대평가, "

    # 종합
    변제정상건["변제내용점검"] = 변제정상건["총분납금점검"] + 변제정상건["총분납회차점검"] + 변제정상건["현재회차점검"] + 변제정상건["납입회차점검"] + 변제정상건["초과입금점검"]

# 분납비고 삭제건 작성
if True :
    # 분납비고 삭제건 각 항목 정상인데, 항목 오류무시구문 있는 경우
    변제내용오류cond = [
        (변제정상건["총분납금점검"]!=""), 
        (변제정상건["총분납회차점검"]!=""),
        (변제정상건["현재회차점검"]!=""), 
        (변제정상건["납입회차점검"]!=""), 
        (변제정상건["초과입금점검"]!=""), 
        (변제정상건["변제내용점검"]!="")
    ]
    변제내용오류무시cond = [
        변제정상건["분납비고"].str.contains(총분납금오류무시구문),
        변제정상건["분납비고"].str.contains(변제내용오류무시구문[0]),
        변제정상건["분납비고"].str.contains(변제내용오류무시구문[1]),
        변제정상건["분납비고"].str.contains(변제내용오류무시구문[2]),
        변제정상건["분납비고"].str.contains(변제내용오류무시구문[3]),
        변제정상건["분납비고"].str.contains(항목오류무시구문+'|'+이달오류무시구문, regex=True)
    ]

        # 변제정상건에 분납비고 삭제할 항목 넣기, 겹칠수 있으므로 select 안됨
    변제정상건["분납비고삭제항목"] = ""
    변제정상건.loc[~변제내용오류cond[0] & 변제내용오류무시cond[0], "분납비고삭제항목"] += "총분납금, "
    변제정상건.loc[~변제내용오류cond[1] & 변제내용오류무시cond[1], "분납비고삭제항목"] += "총분납회차, "
    변제정상건.loc[~변제내용오류cond[2] & 변제내용오류무시cond[2], "분납비고삭제항목"] += "현재회차, "
    변제정상건.loc[~변제내용오류cond[3] & 변제내용오류무시cond[3], "분납비고삭제항목"] += "납입회차, "
    변제정상건.loc[~변제내용오류cond[4] & 변제내용오류무시cond[4], "분납비고삭제항목"] += "초과입금, "
    변제정상건.loc[~변제내용오류cond[5] & 변제내용오류무시cond[5], "분납비고삭제항목"] += "변제내용관련전체, "

        # 분납비고 삭제할 행만 추리기
    변제내용점검정상 = 변제정상건[변제정상건["분납비고삭제항목"]!=""][["분납키","채무자키","계좌키","분납비고","분납비고삭제항목","분납메모","분납기타","기타금액"]]
    변제내용점검정상.rename(columns={"분납비고":"구분납비고"},inplace=True)
    변제내용점검정상["분납비고"] = "NULL"
    변제내용점검정상 = 변제내용점검정상[["분납키","채무자키","계좌키","분납비고","구분납비고", "분납비고삭제항목","분납메모","분납기타","기타금액"]]

# 오류건 작성
if True :
    # 오류건 : 오류무시구문 따라 오류구문 지워서 다시 종합
        # 전체무시건 모든 항목 오류구문 지우기
    변제정상건.loc[변제내용오류무시cond[5], ["총분납금점검", "총분납회차점검", "현재회차점검", "납입회차점검", "초과입금점검"]] = ""
        # 항목별로 지우기
    변제정상건.loc[변제내용오류무시cond[0], "총분납금점검"] = ""
    변제정상건.loc[변제내용오류무시cond[1], "총분납회차점검"] = ""
    변제정상건.loc[변제내용오류무시cond[2], "현재회차점검"] = ""
    변제정상건.loc[변제내용오류무시cond[3], "납입회차점검"] = ""
    변제정상건.loc[변제내용오류무시cond[4], "초과입금점검"] = ""
        # 종합열 다시 작성, 총분납금열은 추가하지 않는다.
    변제정상건["변제내용점검"] = 변제정상건["총분납회차점검"] + 변제정상건["현재회차점검"] + 변제정상건["납입회차점검"] + 변제정상건["초과입금점검"]

        # 최종정의
            # 총분납금은 출력칼럼이 다르니 따로 저장
    총분납금점검오류 = 중간회차.loc[변제정상건[변제정상건["총분납금점검"] != ""].index]
    변제내용점검오류 = 변제정상건[변제정상건["변제내용점검"] != ""]

    # 칼럼 정의
    중간회차점검cols = 기본조회칼럼 + ["초과입금"] + 회차보조칼럼 + 전산회차칼럼 + 법원회차칼럼 + 변제분석칼럼 + ["OPB-총분납금", "OPB", "총분납금", "분납컷오프일", "신고조회일", "수정된당사시작회차", "수정된분납컷오프일", "현재당사시작회차", "매각사"]
    변제내용점검cols = 기본조회칼럼 + ["변제내용점검", "초과입금"] + 회차보조칼럼 + 전산회차칼럼 + 법원회차칼럼 + 변제분석칼럼 + ["OPB-총분납금", "OPB", "총분납금", "분납컷오프일", "신고조회일","매각사"]

##### 출력

In [128]:
# 출력
if len(총분납금점검오류) > 0 :
    print(f'중간회차, (OPB>총분납금) 확인필요건 {len(총분납금점검오류)}건 엑셀파일 출력')
    functions.save_df_to_excel_underline(총분납금점검오류[중간회차점검cols], join(path_wd_check, "[check]개인회생_3-1.변제내용점검_중간회차_총분납금_"+cutoff+".xlsx"), key_columns_no=6)
else : 
    print('중간회차, (OPB>총분납금) 확인필요건 없음')

if len(변제내용점검오류) > 0 :
    변제내용점검오류_초회 = 변제내용점검오류[변제내용점검오류["초회여부"]=="초회"]
    변제내용점검오류_중간 = 변제내용점검오류[변제내용점검오류["초회여부"]!="초회"]
    print(f'변제내용점검오류 : 초회 {len(변제내용점검오류_초회)}건, 중간{len(변제내용점검오류_중간)}건 엑셀출력.')
    functions.save_df_to_excel_underline(변제내용점검오류_초회[변제내용점검cols], join(path_wd_check, "[check]개인회생_3-2.변제내용점검_초회_"+cutoff+".xlsx"), key_columns_no=6)
    functions.save_df_to_excel_underline(변제내용점검오류_중간[변제내용점검cols], join(path_wd_check, "[check]개인회생_3-2.변제내용점검_중간_"+cutoff+".xlsx"), key_columns_no=6)
else : 
    print('변제내용 점검할 건이 없습니다.')
    
if len(변제내용점검정상) > 0 : 
    print(f'분납비고 삭제 체크 : {len(변제내용점검정상)}건. 엑셀출력. 초과입금은 거의 삭제필요 없을것')
    functions.save_df_to_excel_underline(변제내용점검정상, join(path_wd_check, "[전산수정]개인회생_3.변제내용점검정상_분납비고삭제_"+cutoff+".xlsx"), key_columns_no=0)
else : 
    print('분납비고 삭제할 건 없음')


중간회차, (OPB>총분납금) 확인필요건 없음
변제내용점검오류 : 초회 37건, 중간64건 엑셀출력.
분납비고 삭제 체크 : 3건. 엑셀출력. 초과입금은 거의 삭제필요 없을것


#### 4.분납상태 중단으로 변경할 건
- 분납상태 진행으로 바꿀것 : 중단 & ~매각,환매 & 최근1개월 개인회생 입금조회

In [110]:
분납상태중단할건 = raw_data.query('분납상태 == "진행" and (상태및담당자.str.contains(@종료) or 상태및담당자.str.contains(@종결요청))')
if len(분납상태중단할건)> 0 : 
    print(f"분납상태 중단으로 변경할 것이 있습니다. {len(분납상태중단할건)}건")
    분납상태중단할건.to_excel(join(path_wd_check, "[check]개인회생_4.분납상태중단할것_"+cutoff+".xlsx"), index=False)
else : 
    print("분납상태 중단할 게 없습니다.")

분납상태 중단으로 변경할 것이 있습니다. 705건


### 점검 후 칼럼 정리

In [14]:
# 점검 끝 : 일반내용 ~ 필터제출기준현재회차 내용삭제
ind1 = raw_data.columns.get_loc('일반내용')
ind2 = raw_data.columns.get_loc('필터제출기준현재회차')
raw_data.iloc[:,ind1:ind2] = ""

# 마지막 변제금, 일시변제,재산처분금액 추가
raw_data.loc[미종결인가건.index, ["마지막회차변제금", "일시변제금액", "재산처분금액"]] = 미종결인가건[["마지막회차변제금", "일시변제금액", "재산처분금액"]]

### 특이건 회차,금액 정리

#### 총분납잔금 - 초과입금(미등록계좌,선입금)
- (241026)모든 초과입금 raw_data에서 없어지게 처리 완료. 초과입금무시건은 선입금이거나 총분납금에 반영완료된 건. 즉 [총분납잔금 = 총분납금 - 총분납입금] 등식 성립
- 분납컷오프 오류 아직 발견하지 못한 건, 매입후변제금상승하는 건은 언제든 초과입금 발생할 수 있음.
- 따라서 오류무시구문 없는 초과입금 발생하면 총분납입금에서 제외하는 것이 맞음.(DC작업해서 초과입금 없애야 하는 것들)

In [27]:
# 1) 총분납금 : 추가할 건 위에서 했다. 수정할 거 없음
# 2) 총분납입금 : 초과입금을 포함한다. 무시건을 제외하고는 총분납입금에서 초과입금을 빼주어야 한다.
# 변제정상건에서 오류무시구문 없는 초과입금은 초과입금점검열이 ""이 아니다.
초과입금ind = 변제정상건[변제정상건["초과입금점검"]!=""].index
print("초과입금 발생건. 오입금확인 건 빼고는 없어야 함")
display(변제정상건.loc[초과입금ind][기본조회칼럼 + 핵심금액칼럼 + ["기타금액", "초과입금"]])

선작업 했다면 없어야 맞음.(오입금 하나 있을 수 있음)
결과 있다면 다시 확인 하고 처리하자. 찍힌 초과입금보다 더 많은 금액을 회생폐지로 전산수정했다면 전산 다시 다운로드 했었어야


,채무자키,계좌키,분납키,분납자관계,분납자성명인,주민번호인,관할법원,사건번호,인가미인가,현재결과,분납메모,분납기타,분납비고,기타금액,총분납금,총분납입금,총분납잔금,기타금액,초과입금
8127,20526072,201059737,20136298,채무자,이정백,851105-1801311,대구지방법원,2019개회223435,인가,인가,,,,0,1420198,1841881,54623,0,476306
6727,20514774,201045627,20130834,채무자,김영준,810418-1691414,대구지방법원,2020개회21994,인가,인가,,,,0,2458336,2450450,72304,0,64418
5362,20514738,201045591,20131298,채무자,조창호,730923-1788611,대구지방법원,2020개회233933,인가,인가,,,,0,11173995,7564181,3724665,0,114851
8417,20410885,200976226,20104599,채무자,설수현,810516-2726715,대전지방법원,2019개회18350,인가,인가,,,,0,7752556,7932848,0,0,180292
3403,20527415,201061080,20136573,채무자,김민우,871129-1408836,대전지방법원,2021개회118222,인가,인가,,,,0,7588573,4305094,3390639,0,107160
2393,20526837,201060502,20137799,채무자,배기상,840213-1408314,대전지방법원,2021개회141659,인가,인가,,,,0,5615652,5057271,601677,0,43296
7289,20410492,200975476,20105271,채무자,권은정,790409-2536315,부산회생법원,2020개회106436,인가,인가,,,,0,1427356,1427356,0,0,0
8289,20411109,200977334,20104577,보증인,김성연,340-86-00348,서울회생법원,2019개회1062450,인가,인가,,"매입OPB_13,858,908",,0,15586536,15770995,371108,0,555567
9108,20500389,201028067,20127653,채무자,한동진,890818-1081720,수원회생법원,2017개회1029273,인가,인가,,,,0,844008,984676,0,0,140668
8335,20410817,200976143,20104510,채무자,곽경아,841220-2226315,수원회생법원,2019개회36166,인가,폐지취소결정송달,,,,0,4141410,4242420,0,0,101010


In [28]:
raw_data.loc[초과입금ind, "총분납입금"] -= raw_data.loc[초과입금ind, "초과입금"]
# 3) 총분납잔금 : 총분납금에 영향을 받으므로 재작성 필요
raw_data["총분납잔금"] = raw_data["총분납금"] - raw_data["총분납입금"]
# +) 분납미납금,분납잔금 : 초과입금때문에 과소평가, 과대평가 된다. 그러나 (특히 미등록채권은) 정확한 금액 산출은 스케줄을 완전히 재작성해야 하므로
    # 고치기 힘들다. 초과입금 몇건 안 되고, 소액이니 오류상태 그대로 둔다. 참고열
# 초과입금) 참고열로 삭제하지 않고 그냥 둔다. 계산에는 반영하지 않는다. 자산정리파일에 원래부터 출력 안 하고 있었다.

# 총분납잔금 음수있는지 확인
총분납잔금음수 = raw_data.loc[미종결인가건.index].query('총분납잔금<0')[기본조회칼럼 + 핵심금액칼럼 + ["기타금액", "초과입금"]]
if len(총분납잔금음수) > 0 : 
    print('총분납잔금 음수 발생!!')
    display(총분납잔금음수)
else : 
    print('총분납잔금 음수 없음')

총분납잔금 음수 없음


#### 통합신고,제외 : 0원처리

In [29]:
# 통합신고, 제외
# 통합신고는 애초에 하나의 계좌로 금액을 합산하여 신고한 경우로 신고채권번호도 한 개. 
#       단일채무자키인 경우 전산 통합등록하고, 아무계좌에서나 변제처리 되어도 상관없다. - 통합등록한 경우 계좌 분리과정에서 대표계좌만 금액이 남는다.
#       다채무자키인 경우 통합등록은 불가, 따라서 대표가 아닌 계좌의 분납메모에는 반드시 통합신고 적어줘야 금액 0원처리 가능
# 제외는 계좌별로 신고하였으나, 원금이 적어 총분납금이 0원인 채권으로 신고채권번호는 여러개이나, 법원채권번호는 하나만 나온 경우
#       신규건은 분리등록이 원칙. 이미 통합등록이 된 경우 분리등록해줘야 하나 그대로 두고 제외가 아닌 통합신고로 처리해도 별 문제는 없음
####### 통합신고채권(채권번호 하나)은 계좌별로 등록할 필요가 없으니 이렇게 적을 필요가 없음(아래서, 계좌통합건은 행분열하고 모두 0원으로 처리, 현재원금은 유지)
####### 통합신고채권은 계좌별로 분리하지 말고 통합해서 전산등록해야

#240131 정정, 회차 및 회차별 변제예정액은 그대로 뒀다. 분납메모만 봐도 되니까.
raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["총분납금","총분납입금", "분납미납금", "분납잔금", "총분납잔금"]] = 0 # 입금내역 있는 것 재입금처리 해야함..
# 공채권류 확인
raw_data.loc[raw_data.분납메모.str.contains("(공채권|통합|제외)"),["채무자키","분납키","주민번호인", "분납메모", "총분납금","총분납입금", "분납미납금", "분납잔금", "총분납잔금"]].info() #.to_excel(join(path_wd, "공채권.xlsx"))

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 322 to 7902
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   채무자키    59 non-null     object
 1   분납키     59 non-null     object
 2   주민번호인   59 non-null     object
 3   분납메모    59 non-null     object
 4   총분납금    59 non-null     int64 
 5   총분납입금   59 non-null     int64 
 6   분납미납금   59 non-null     int64 
 7   분납잔금    59 non-null     int64 
 8   총분납잔금   59 non-null     int64 
dtypes: int64(5), object(4)
memory usage: 4.6+ KB


#### 연납건
- 분납메모가 있어야만 작성할 수 있다.
- 분납메모작성법 : 일반회생_당사에납입한회차_구간정보1(시작회차~종료회차:변제금액),구간정보2,... (매입전 납입한회차는 매입전변제금에 작성)
  - ex) 일반회생_3_1~9:41399, 10:41395

In [30]:
# 연납건 중 인가건 처리-----------------------------------------------------------------------------------------------------------------------------------
# 나우리의 계산법과 다르게 컷오프전 입금회차를 포함함.(즉 언제나 1회차 부터계산)
연납건index = raw_data.query('분납메모.str.contains("일반회생") and 분납상태=="진행"').index

if re.search("\d{6,6}",cutoff) :
    real_cutoff = cutoff
else : real_cutoff = datetime.strftime(datetime.today(), "%y%m%d")

for i in 연납건index : 

    기본정보 = raw_data.loc[i,"분납메모"].split("_")
    시작일 = datetime.strptime(raw_data.loc[i,"분납시작일"], "%Y-%m-%d")
    종료일 = datetime.strptime(raw_data.loc[i,"분납종료일"], "%Y-%m-%d")

    # 회차
    총분납회차 = 종료일.year - 시작일.year + 1
    if real_cutoff[2:] < str(시작일.month)+str(시작일.day) : 
        현재회차 = int("20"+real_cutoff[:2]) - 시작일.year # ex) 2022년 9월 30일, 분납시작일 2021년 12월 1일 > 1회차
    else : 
        현재회차 = int("20"+real_cutoff[:2]) - 시작일.year + 1 # ex) 2022년 12월 30일, 분납시작일 2021년 12월 1일 > 2회차
    납입회차 = int(기본정보[1]) + raw_data.loc[i,"매입전변제금"] # 솔림입금회차 + 매입전 입금회차
    잔여회차 = 총분납회차 - 현재회차
    미납회차 = 현재회차 - 납입회차

    # 금액
    _회차별_변제금_ = 기본정보[-1].split(",")

    회차별_변제금 = pd.Series(index=range(1,총분납회차+1), name="회차별변제금")
    for v in _회차별_변제금_ :
        회차구간, 회차구간별변제금 = v.split(":")
        회차구간별변제금 = int(회차구간별변제금)

        if re.search("~", 회차구간) : 
            sn = int(회차구간.split("~")[0])
            en = int(회차구간.split("~")[1])
            회차별_변제금.loc[sn:en] = 회차구간별변제금
        else : 
            회차별_변제금.loc[int(회차구간)] = 회차구간별변제금

    총분납금 = 회차별_변제금.sum()
    총분납입금 = 회차별_변제금.loc[:납입회차].sum()
    총분납잔금 = 총분납금 - 총분납입금 # = 분납잔금 + 분납미납금 
    분납잔금 = 회차별_변제금.loc[현재회차+1 : ].sum() # 미래
    분납미납금 = 총분납금 - 총분납입금 - 분납잔금 # 과거 # 구간별 변제금이 다를 수 있으므로 미납회차*변제금을 할 수 없음

    raw_data.loc[i, "총분납회차"], raw_data.loc[i, "현재회차"], raw_data.loc[i, "납입회차"], raw_data.loc[i, "잔여회차"], raw_data.loc[i, "미납회차"] = 총분납회차, 현재회차, 납입회차, 잔여회차, 미납회차
    raw_data.loc[i, "총분납금"],raw_data.loc[i, "총분납입금"], raw_data.loc[i, "분납미납금"], raw_data.loc[i, "분납잔금"], raw_data.loc[i, "총분납잔금"] = 총분납금, 총분납입금, 분납미납금, 분납잔금, 총분납잔금


### 반영OPB 및 요약 : 금액칼럼 수정 후에 위치할것

In [36]:
raw_data['반영OPB'] = np.where(raw_data["인가미인가"]=="인가", raw_data["총분납잔금"], raw_data["현재원금"] + raw_data["현재법비용"])
# 240613이후 raw_data['반영OPB'] = np.where(raw_data['인가미인가'] == '인가', raw_data["총분납잔금"], raw_data["현재원금"] + raw_data["현재법비용"])
# 240613이전 raw_data['반영OPB'] = np.where(raw_data['인가미인가'] == '인가', raw_data["총분납잔금"]-raw_data["초과입금"] , raw_data["현재원금"] + raw_data["현재법비용"])
# OPB : 현재법비용 추가일 2024-04-03

# 오류 확인 : 입금 없었는데, 분납잔금도 없는 건
# 미인가라면 제외처리해야하고, 인가라면 인가잔액 적어줘야
display(raw_data.query('분납상태 == "진행" and 인가미인가.str.contains("인가|미인가") and ~상태및담당자.str.contains(@종료) \
    and ~상태및담당자.str.contains(@종결요청) & ~분납메모.str.contains("제외|통합") and 반영OPB==0 and 총분납입금==0')[기본조회칼럼])


# # 요약
# 전체요약 = raw_data.query('분납상태=="진행"').groupby(['인가미인가','현재결과']).agg({'분납키':'count', '반영OPB':'sum'})
# 전체요약.loc[("합계", ""), :] = 전체요약.sum()
# 전체요약.applymap('{:,.0f}'.format)

,채무자키,계좌키,분납키,분납자관계,분납자성명인,주민번호인,관할법원,사건번호,인가미인가,현재결과,분납메모,분납기타,분납비고,기타금액
268,20514623,201045476,20148576,채무자,김동훈,900324-1031313,서울회생법원,2023개회1199311,인가,인가,,,인가신고_조회결과_별제권but확정신고미실시,0


### 진행건 분리 - 종결건 제외하지 않아야 함

In [30]:
# 불필요한 열 삭제 - 이미 삭제한 열이 있어도 오류나지 않는 방식
# 삭제할 열 목록
columns_to_drop = ['상태및담당자']
# 남길 열 목록을 원래의 순서대로 계산
columns_to_keep = [col for col in raw_data.columns if col not in columns_to_drop] # difference 함수 쓰면 원래 순서가 엉망으로 바뀌어버림
# 필요한 열만 남기기
raw_data = raw_data[columns_to_keep]

# 진행중, 차주, 보증인 시트 작성(종결인데 분납중지 안 된 것 여기선 제외x)
ing_df = raw_data.query('분납상태 == "진행" and 인가미인가.str.contains("인가|미인가")')

# 종료건 제외 : 일부 계좌만 종결|매각|환매 될 수 있으므로 여기서 제외하지 않는다. -> 종료건 제외도 계좌별 상태가 확정되고 해야 함
# 어차피 통합에선 종결건은 우선 제외하므로 자산정리에 문제는 없음
# 통합파일에서 계좌별로 상태 확인 후 종료인데 중지 안 된 건 처리
# 최초 작성코드
# if path_wd.find("대성") < 0 : # 솔림
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각(?!예정)|DS|환매"))]
# else : # 대성
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각|환매|파산|회계파산"))]

    
###### 신복과 충돌하는 거 제거하기 #############################################################
if path_wd.find("대성") < 0 : # 솔림
    개회제외 = {"채무자" : [], "보증인" : [] } # 20462830 중복입금건. 신복으로 처리하다가 개인회생 면책됨 231106
else : # 대성
    개회제외 = {"채무자" : [], "보증인" : [] } # 
##############################################################################################
if len(개회제외["채무자"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["채무자"]) & (ing_df['분납자관계'] == '채무자'))]
if len(개회제외["보증인"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["보증인"]) & (ing_df['분납자관계'] == '보증인'))]

# 채무자, 보증인
debt_ing = ing_df[ing_df["분납자관계"]=="채무자"]
grt_ing = ing_df[ing_df["분납자관계"]=="보증인"]

# 저장
with pd.ExcelWriter(join(path_wd, name_dataset+"_기준데이터.xlsx"), engine='openpyxl', mode = 'w') as writer :
    ing_df.to_excel(writer, sheet_name='개인회생(진행)', index=False)
    debt_ing.to_excel(writer, sheet_name='개인회생(진행)_채무자', index=False)
    grt_ing.to_excel(writer, sheet_name='개인회생(진행)_보증인', index=False)
    raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

##### 계좌키 나누고 새로운 파일 저장

In [31]:
# 개인회생 계좌 나누기

# path_wd = r"D:\3.자산\전산 dataset\230430(0501)"
# name_dataset = "개인회생새창_20230502_0850_기준데이터"
# ing_df = pd.read_excel(join(path_wd, name_dataset+".xlsx"), sheet_name="개인회생(진행)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

if len(ing_df.계좌키.apply(len).value_counts()) > 1 : # 계좌키의 길이값이 두 종류 이상이라면(통합계좌 있으면)

    split_ing_df = ing_df.copy()
    split_ing_df["비고"] = ""
    new_rows = []
    for index, row in split_ing_df.iterrows():
        account_keys = row['계좌키'].split(',')
        first_account_key = account_keys.pop(0)
        split_ing_df.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

        for account_key in account_keys:
            new_row = row.copy() # 일단 모든 칼럼 복사
            new_row['계좌키'] = account_key
                                # 전산 OPB는 최초OPB임
            new_row['반영OPB'],new_row['OPB'], new_row['총분납금'],new_row['총분납입금'],new_row['분납미납금'],new_row['분납잔금'],new_row['총분납잔금'] = 0,0,0,0,0,0,0 # 값 안 가져올 컬림은 빈문자열로 대체
            new_row["분납메모"] = "통합신고 " + first_account_key
            new_rows.append(new_row)
            

    temp = pd.DataFrame(new_rows)

    result = pd.concat([split_ing_df, temp], ignore_index=True)

    # 저장
    writer = pd.ExcelWriter(join(path_wd, name_dataset+"_진행중_계좌키분리_기준데이터.xlsx"))
    result.to_excel(writer, sheet_name="개인회생(진행)", index=False)
    # writer.save() # 이제 save()는 없어진 api 안 쓰면 됨.
    writer.close()


### [DC] CHECK

#### 계좌 분리 후 체크할 것

In [32]:
# 면책건

# cols = ["채무자키", "계좌키", "분납자관계", "분납자성명인", "인가미인가", "현재결과", "분납상태", "분납중단일", "중단사유","현재결과일","면책결정일"]
면책 = raw_data.query('인가미인가=="면책" or 중단사유 in ["면책","납부완료","중도완제"] or 면책결정일!=""').copy()

# 계좌키를 쉼표로 분리하여 여러 개의 행으로 분할
splitted_df = pd.DataFrame(면책['계좌키'].str.split(',').tolist(), index=면책.index).stack() # 스택함수를 쓰면 열이 행으로 분리되고, level_0(원래 인덱스), level_1 의 이차원 인덱스가 생긴다.

# 인덱스 재설정
splitted_df = splitted_df.reset_index(level=1, drop=True).rename('분리계좌키') # 레벨0만 남기고 1은 드랍

# 합치기
면책_행분리 = 면책.merge(splitted_df, how='left', left_index=True, right_index=True)

면책_행분리.drop(['계좌키'],axis=1, inplace=True)
면책_행분리.insert(6,"분리계좌키", 면책_행분리.pop("분리계좌키"))
면책_행분리.rename(columns={"분리계좌키" : "계좌키"}, inplace=True)

# 면책_행분리.to_excel(join(path_wd, "[check]개인회생_면책건_계좌키분리_"+cutoff+".xlsx"), index=False)

In [33]:
# 보증인새창조회
####################################################
wd = r"D:\3.자산\전산 dataset\솔림\240930"
grt_ori = pd.read_excel(join(wd, "보증인새창_20241002_1033.xlsx"), dtype=grt_dtype).fillna("")
####################################################

#### 개인회생과 차주상태

In [34]:
# 면책건
차주면책건 = 면책_행분리.query('분납자관계=="채무자"')
보증인면책건 = 면책_행분리.query('분납자관계=="보증인"')

# 면책종결 관련 단어
면책종결 = r"종결|종료|면책|면탈|완납|완제|매각(?!예정)|(?<!피)환매|DS"

# 조회할 칼럼
cols = ["채무자키", "계좌키", "분납키", "분납자관계", "분납자성명인", "인가미인가","현재결과","분납메모","분납비고","분납중단일","중단사유","면책결정일","채무상태","담당자","주민번호인"]

In [ ]:
# 차주상태 불일치건(기준일 면책건), 누락채권은 누락이 맞음. 
# 동일주민번호에 따른 채무상태 일치여부는 따로 확인하므로 개인회생데이터만봐도 됨.
check차주면책 = 차주면책건.query('~(채무상태.str.contains(@면책종결) or 담당자.str.contains(@면책종결))')[cols]
if len(check차주면책) > 0 :
    print(f"차주 개회 면책인데 전산 상태및담당자는 그렇지 않은 건 : {len(check차주면책)}건")
    check차주면책.to_excel(join(wd,"[check]최근면책건_차주.xlsx"), index=False)
else :
    print("차주 면책 상태및담당자 이상무")
# 누락건 제외하고는 계좌조회새창의 채무상태를 개인회생(면책)으로.

#### 개인회생과 보증인상태

In [36]:
grt = grt_ori.copy()

In [ ]:
# 상태 '종결' 제외
grt1 = grt.query('~보증인상태.str.match("종결|보증면탈") and ~(담당자.str.contains(@종료))') # 종결건은 빼고 살핀다.

# 언더바 제거
grt1["보증인상태"] = grt1.보증인상태.str.replace("_", "")

# 일단 보증인 상태값에 불일치 값이 없는지 체크 # 채무상태는 아쉽게도 없네
보증상태값중복 = grt1.groupby(by=["주민번호인"]).filter(lambda x: x.보증인상태.nunique() != 1)[["채무자키","채무자주민번호", "계좌키", "보증인키", "보증인성명인", "주민번호인", "보증인상태","담당자"]].sort_values(by=["주민번호인"])
if len(보증상태값중복) : 
    print(f"미종결 채권의 동일 보증인상태가 다른건 : {len(보증상태값중복)}건. 엑셀 확인")
    보증상태값중복.to_excel(join(wd, "[check]보증상태값중복.xlsx"), index=False)
else : 
    print("미종결 채권의 보증인 상태값 불일치 이상무 ---")

In [ ]:
# 보증인상태 불일치건

# 개인회생새창에는 보증인 주민번호가 없으므로 보증인 주민번호를 먼저 찾아야함.
merged1 = 보증인면책건[cols].merge(grt[["채무자키", "보증인성명인", "주민번호인"]], left_on=["채무자키", "분납자성명인"], right_on=["채무자키", "보증인성명인"], how='left', \
    suffixes=("","_보증인")).drop(["보증인성명인"], axis=1)

# 인가미인가와 보증인 상태를 비교
merged2 = merged1.merge(grt[["주민번호인", "보증인상태"]], left_on=["주민번호인_보증인"], right_on=["주민번호인"], suffixes=("_채무자","_y"), how='left').drop(["주민번호인_y"], axis=1)

# 보증인 개회면책인데 보증인상태가 종결류가 아닌 것
merged3 = merged2.query('~보증인상태.str.contains("종결|면책|면탈|완납", na=False) and ~담당자.str.contains("종결")')

if len(merged3) > 0 :
    print(f"차주 개회 면책인데 전산 상태및담당자는 그렇지 않은 건 : {len(merged3)}건")
    merged3.to_excel(join(wd, "[check]보증인상태면책수정.xlsx"), index=False)
else : 
    print("보증인 면책 상태및담당자 이상무")

In [ ]:
merged2.query('주민번호인_보증인.isin(@보증상태값중복.주민번호인)')

### (필요할때만)수동입금 대상 : 사건번호(주민번호x)동일, 변제액 동일, 배수관계
- 입금파일에는 사건번호만 있으므로 사건번호로만 그룹화
- 다회차가 들어와서 배수관계가 되는 건은 특정할 수 없으므로 사후확인

In [ ]:
def has_multiple_relationship(series):
    """현재회차변제금들이 서로 배수 관계인지 확인"""
    non_zero_values = [x for x in series if x > 0]
    if len(non_zero_values) < 2:
        return False
    sorted_series = sorted(non_zero_values)
    
    # 각 값을 base로 시도
    for i in range(len(sorted_series)-1):
        base = sorted_series[i]
        for other in sorted_series[i+1:len(sorted_series)]:
            if other % base == 0 : 
                return True
    return False

변제정상건["현재회차변제금"].replace("", np.nan, inplace=True)
변제정상건["현재회차변제금"].fillna(0, inplace=True)
변제정상건["현재회차변제금"] = 변제정상건["현재회차변제금"].astype(int)

grouped = 변제정상건.groupby(by=["사건번호"])

# 배수 관계에 있는 사건번호 찾기
multiples = grouped['현재회차변제금'].apply(has_multiple_relationship)
multiples_cases = multiples[multiples].index.tolist()


# 엑셀 조회
배수cols = 기본조회칼럼 + 전산회차칼럼 + 법원회차칼럼 + 회차보조칼럼 + ["변제채권번호입력"]
변제금배수 = 변제정상건[변제정상건['사건번호'].isin(multiples_cases)][배수cols]
if len(변제금배수)>0:
    print(len(변제금배수), "건의 변제금배수건 발견")
    functions.save_df_to_excel_underline(변제금배수, join(path_wd, "[check]개인회생_변제금배수(입금유의)_"+cutoff+".xlsx"), key_columns_no=6)
else : 
    print("변제금 배수건 없음")

### 불필요함수

In [ ]:
##### 불필요 함수
def is_last_day_of_month(date_str):
    # 문자열을 datetime 객체로 변환
    date = datetime.strptime(date_str, "%Y-%m-%d")
    
    # 해당 월의 마지막 날 계산
    last_day = calendar.monthrange(date.year, date.month)[1]
    
    # 입력 날짜가 그 달의 마지막 날인지 확인
    return date.day == last_day


def 현재회차정보찾기(row) :
    """ 변제구간값들을 받아서 return 채권자변제시작일, 실변제일, 실변제금, 말일여부, 채권자종료회차 """
    # 조회일전일이 말인인 경우만 유의하면 됨. 예컨대 start이 1/31인데 
    # 조회일이 3/1이면 +1되는 게 맞고, 2/28이면 
    # -1, -2:분납후? 어쨌든 구간 마지막에 나옴
    
    채권자변제시작일, 실변제일, 실변제금, 말일여부, 구간번호, 채권자종료회차, 채권자변제일미도래, 채권자완납여부 = "", "", "", "", 0, 0, False, False # 기본값 : False
    
    # 값 없음[시작] ---------------------------------------------------
    
    # 변제금합계가 없는 경우(총분납금은 제외채권이라도 이미 읽어졌을 수도 있음)
    # 제외채권처리 했다면 사실 이 경우는 없는 경우
    if row["변제금합계"] == 0 :
        return 채권자변제시작일, 실변제일, 실변제금, 말일여부, 채권자종료회차
    
    # 변제조회 실패, 조회성공했다면 기간1에는 무조건 값이 있다.
    if row['변제기간1'] == "" : 
        return 채권자변제시작일, 실변제일, 실변제금, 말일여부, 채권자종료회차
    
    # 값 없음[끝] ---------------------------------------------------
    
    # 변제조회 성공
    else : 
        # 구간순회하며 법원납입회차(현재회차 대신) 에 맞는 정보찾기
        
        # 1. 먼저 채권자변제시작일 찾기. 반복분 따로 돌려야 오류 없겠다.
        for i in range(1, 11):
            if row[f'변제예정액{i}'] > 0 :
                채권자변제시작일 = row[f'변제기간{i}'][:10]
                채권자변제시작회차 = int(row[f'변제회차{i}'].split("~")[0])
                break
        
        
        # 1-1. 당사 변제일 미도래
        if datetime.strptime(채권자변제시작일,"%Y-%m-%d") > cutoff_dt :
            # 240705 현재 인가&변제조회 정상건 중 변제기일,구간별납입일자,실변제일이 서로 다른 경우는 말일인 경우밖에 없없음.
            # 결국 실변제일은 안 구해도 됐음...일단 혹시 모르니 그냥 두고, 변제기일을 출력하자.
            실변제일 = row["변제기일"] 
            if len(실변제일) == 2 : # 전산도 에러 아니면, 이 조건 달아야 아래 if문 오류 안날테니
                if is_last_day_of_month(row["신고조회일"][:8]+실변제일) : # 변제조회일(월) 기준으로 전산이 변제기일 잡을 것이므로
                    말일여부 = "말일"
            return 채권자변제시작일, 실변제일, 실변제금, 말일여부, 채권자종료회차
        
        
        # 2. 채권자종료회차 찾기 -  미도래건 다음에
        
        법원납입회차 = row['법원납입회차']
        # 채권자변제시작회차부터 시작해야함
        if 법원납입회차 < 채권자변제시작회차 : 
            법원납입회차 = 채권자변제시작회차
        
        # 현재회차 = int(row['현재회차'])
        
        for i in range(1,11) :
            if row[f'변제회차{i}'] == "" : 
                마지막회차구간 = row[f'변제회차{i-1}'] #i-1
                if '~' in str(마지막회차구간) :
                    start, end = map(int, 마지막회차구간.split('~'))
                    채권자종료회차 = end
                else : 
                    채권자종료회차 = int(마지막회차구간)
                break
                
        # 2-1 채권자변제종료
        if 법원납입회차 >= 채권자종료회차 :
            return 채권자변제시작일, 실변제일, 실변제금, 말일여부, 채권자종료회차
        
        # <수정전> 2-1 # 현재회차 기준 전체 변제 종료
        # if datetime.strptime(row["변제종료일"],"%Y-%m-%d") <= cutoff_dt :
        #     실변제일 = row["변제기일"] 
        #     if len(실변제일) == 2 : # 전산도 에러 아니면, 이 조건 달아야 아래 if문 오류 안날테니
        #         if is_last_day_of_month(row["신고조회일"][:8]+실변제일) : # 변제조회일(월) 기준으로 전산이 변제기일 잡을 것이므로
        #             말일여부 = "말일"
        #     return 채권자변제시작일, 실변제일, 실변제금, 말일여부
        
        
        
        
        # 4. 다음으로 실변제일,실변제금,말일여부 찾기
        for i in range(1, 11):
            
            회차값 = row[f'변제회차{i}']
            
            # 회차값 없으면 다음 구간으로
            if 회차값 == "":
                continue
            
            # 현재회차에 맞는 구간번호 찾기
            if '~' in str(회차값):
                start, end = map(int, 회차값.split('~'))
                if start <= 법원납입회차 <= end:
                    구간번호 = i
            else:
                if int(회차값) == 법원납입회차:
                    구간번호 = i
                    
            # 맞는 구간번호 찾았으면 값 확정하고 리턴
            if 구간번호 : # != 0
                실변제금 = row[f'변제예정액{구간번호}']
                현재회차변제시작일 = row[f'변제기간{구간번호}'][:10]
                실변제일 = 현재회차변제시작일[-2:] 
                
                #실변제일(변제시작일의 날짜값)이 말일인지 체크
                if is_last_day_of_month(현재회차변제시작일) :
                    말일여부 = "말일"
                
                return 채권자변제시작일, 실변제일, 실변제금, 말일여부, 채권자종료회차
            else : # == 0
                continue # 반복문 계속
            
        # 반복문 끝나도 뭔가가 잘못됐다면 예측못한 오류
        return "오류", "오류", "오류", "오류", "오류"

def 당사변제시작일찾기(row):
    try:
        실변제일 = int(row['변제기일'])
    except ValueError:
        return ""
    
    reference_date = row['최종컷오프일']
    if isinstance(reference_date, str):
        reference_date = datetime.strptime(reference_date, '%Y-%m-%d')
    
    컷오프년도 = reference_date.year
    컷오프월 = reference_date.month
    컷오프날짜 = reference_date.day
    
    if 컷오프날짜 >= 실변제일:
        컷오프월 += 1
        if 컷오프월 > 12:
            컷오프월 = 1
    
    컷오프월첫날 = datetime(컷오프년도, 컷오프월, 1)
    컷오프후첫변제일 = get_payment_date(컷오프월첫날, 실변제일)
    
    채권자변제시작일 = row['채권자변제시작일']
    if isinstance(채권자변제시작일, str):
        채권자변제시작일 = datetime.strptime(채권자변제시작일, '%Y-%m-%d')
    
    return max(컷오프후첫변제일, 채권자변제시작일).strftime('%Y-%m-%d')
    
 
def 최근변제예정일찾기(row, reference_date):
    """ reference_date는 날짜거나 %Y-%m-%d형태의 문자열일것"""
    try : 
        실변제일 = int(row['변제기일']) # 실변제일에서 변제기일로 바꿈
    except ValueError: # 실변제일 없는 경우
        return ""
    if isinstance(reference_date, str) :
        reference_date = datetime.strptime(reference_date,'%Y-%m-%d')
    조회년도 = reference_date.year
    조회월 = reference_date.month
    
    # 참조일 : 현재월의 1일(1일은 어느 달이나 있으니까)
    참조일 = datetime(조회년도, 조회월, 1)
    참조일이전달 = 참조일 - relativedelta(months=1)
    참조일다음달 = 참조일 + relativedelta(months=1)
    
    # 이번달, 지난달, 다음달 변제일 계산
    이번달변제일 = 말일오류없애기(참조일, 실변제일)
    이전달변제일 = 말일오류없애기(참조일이전달, 실변제일)
    다음달변제일 = 말일오류없애기(참조일다음달, 실변제일)
  
    
    # 조회일과 가장 가까운 날짜 계산
    최근변제예정일 = min([이번달변제일, 이전달변제일, 다음달변제일], 
                       key=lambda date: abs(참조일 - date))
    
    return 최근변제예정일.strftime('%Y-%m-%d')
   
   


    
def save_df_to_excel(df, filename, font_size=9):
    "날짜타입은 문자열로 바꿔주고, 글씨크기 조정한다."
    # DataFrame을 엑셀 파일로 저장
    df.to_excel(filename, index=False, sheet_name='Sheet1')

    # 엑셀 파일 열기
    workbook = load_workbook(filename)
    sheet = workbook['Sheet1']
    
    # 날짜 포맷 스타일 정의
    date_style = NamedStyle(name='date_style', number_format='yyyy-mm-dd')
    
    # 날짜 타입인 열에 대해 형식 적용
    for column in df.select_dtypes(include=['datetime64']).columns:
        col_idx = df.columns.get_loc(column) + 1  # Excel은 1-based index
        for row in range(2, sheet.max_row + 1):  # 데이터가 2번째 행부터 시작
            cell = sheet.cell(row=row, column=col_idx)
            cell.style = date_style

    # 글씨 크기 변경 (폰트 설정)
    font = Font(size=font_size)
    
    # 모든 셀에 대해 글씨 크기 설정
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            cell.font = font

    # 엑셀 파일 저장
    workbook.save(filename)